<a href="https://colab.research.google.com/github/namdarine/TIF_Chicago_Project/blob/main/connection_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 0. Import library

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.wkt import loads
import warnings
warnings.filterwarnings("ignore")
import pickle
import folium
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import gamma

# 0. Import data

In [ ]:
JJ_path = '/content/drive/MyDrive/IPRO497_Demographic Team/Final/data/'
K_path = '/Users/kunwoosmac/Library/Mobile Documents/com~apple~CloudDocs/Python Workspace/IIT Courses/TIF DS Project/Final/data/'
L_path = '/content/drive/MyDrive/IPRO/IPRO 497 TIF Districts/IPRO497_Demographic Team/Final/data/'

path = JJ_path

Community_area = gpd.read_file(path + 'Boundaries - Community Areas (current).geojson')

tifs = pd.read_csv(path + '2019_TIF_Boundaries.csv')
tifs = gpd.GeoDataFrame(tifs)
tifs = tifs.rename(columns={'the_geom': 'geometry'})
tifs['geometry'] = loads(tifs['geometry'])
tifs.set_geometry('geometry', inplace = True)

Blockgroup_boundary = pd.read_csv(path+'cleanBlockGroups.csv')
Blockgroup_boundary = Blockgroup_boundary.drop(columns=['coords'])
prefix = '1500000US'
Blockgroup_boundary['geoid'] = prefix + Blockgroup_boundary['geoid'].astype(str)

racial_population = pd.read_csv(path+'racial_population.csv')

# 0. Import pickle files

In [ ]:
# load pickle file
with open(path+'DemographicsDataframe.pickle', 'rb') as f:
	DemographicsDataframe = pickle.load(f)

with open(path+'racial_population.pickle', 'rb') as f:
	racial_population = pickle.load(f)

# **1**. Aggregate metric at communivty level (TIF) - Juhyun Jung


In [ ]:
def aggregate_community_level_TIF(DemographicsDataframe_info,metric,Blockgroup_boundary_info,column_name_info):
  """
  INPUT :
  DemographicsDataframe_info - Ratio/Relationship between CBG, TIF, and community
  metric                     - e.g) income, housing price, owner, ...
  Blockgroup_boundary_info   - Blockgroup boundary
  column_name_info           - the column name of metric e.g) ['Community', 'Less than $10,000', '$10,000 - $14,999', ... ]

  OUTPUT : metric data in TIF at the community level.
  -----------------------------------------------------------------------------------------------------------------------------------------
  test code
  test_incomeDistribution = pd.DataFrame({'geoid':['a','b','c','d','e'],
                      'Less than $10,000':[[10,2,40],[2,4,8],[20,30,70],[40,10,20],[90,80,70]],
                      '$10,000 to $14,999':[[30,40,50],[4,6,10],[50,60,100],[86,40,32],[10,20,30]]})

  test_demogrpahicsDataFrame = pd.DataFrame(columns=['Community','CBG in Community','TIF in Community','CBG in TIF','CBG-Community-TIF Overlap','Non-TIF Ratio of CBG'])
  test_demogrpahicsDataFrame['Community'] = [0,0]
  test_demogrpahicsDataFrame=test_demogrpahicsDataFrame.drop(1)
  test_demogrpahicsDataFrame['Community'].iloc[0] = 'DOUGLAS'
  test_demogrpahicsDataFrame['CBG in Community'].iloc[0] = ['a', 'b', 'c', 'd','e']
  test_demogrpahicsDataFrame['TIF in Community'].iloc[0] = ['x', 'y', 'z']
  test_demogrpahicsDataFrame['CBG in TIF'].iloc[0] = [['a', 'b'], ['a', 'c', 'e'], ['e', 'd']]
  test_demogrpahicsDataFrame['CBG-Community-TIF Overlap'].iloc[0] = [[0.1, 0.2], [0.1, 0.5, 0.2], [0.1, 0.2]]
  test_demogrpahicsDataFrame['Non-TIF Ratio of CBG'].iloc[0] = [0.1, 0.5, 0.1, 0.2,0.3]
  column_name = ['Community','Less than $10,000','$10,000 to $14,999']
  aggregate_community_level_TIF(test_demogrpahicsDataFrame,test_incomeDistribution,Blockgroup_boundary,column_name)


  result
  Community	Less than $10,000	$10,000 to $14,999
	DOUGLAS	[47.4, 42.2, 69.6]	[52.0, 53.2, 77.4]

  -----------------------------------------------------------------------------------------------------------------------------------------
  """

  GEOID = metric.columns[0]  # name of geoid e.g) 'geoid', 'GEO_ID', 'geo_id', 'GEOID'
  community_names = DemographicsDataframe_info['Community']
  no_data_of_geoid = []
  column_number = len(column_name_info)-1 # reason for -1 : 'Community'
  aggregated_TIF = []
  for com_number, com_name in enumerate(community_names):
    lst = np.zeros((column_number,3))
    for range_name_index, range_name in enumerate(column_name_info[1:]):
      CBG_in_TIF_Community = [item for sublist in DemographicsDataframe_info['CBG in TIF'].iloc[com_number] for item in sublist]
      CBG_in_TIF_Community_index = [index for index, sublist in enumerate(DemographicsDataframe_info['CBG in TIF'].iloc[com_number]) for _ in range(len(sublist))]
      for CBG_index,(index, GEOID_TIF) in enumerate(zip(CBG_in_TIF_Community_index,CBG_in_TIF_Community)):
        if len(metric[metric[GEOID]==GEOID_TIF][range_name])==0:
          no_data_of_geoid.append(GEOID_TIF)
        else:
          Overlap_list = [item for sublist in DemographicsDataframe_info['CBG-Community-TIF Overlap'].iloc[com_number] for item in sublist]
          ratio = Overlap_list[CBG_index]
          lst[range_name_index] += np.array(metric[metric[GEOID]==GEOID_TIF][range_name].iloc[0])*ratio
    aggregated_TIF.append(np.array([[com_name]+list(lst)],dtype=object)[0])
  aggregated_TIF_df = pd.DataFrame(aggregated_TIF, columns = column_name_info)
  return aggregated_TIF_df

# **2.** Aggregate metric at communivty level (Non TIF) -Juhyun Jung

In [ ]:
def aggregate_community_level_Non_TIF(DemographicsDataframe_info,metric,Blockgroup_boundary_info,column_name_info):
  """
  INPUT :
  DemographicsDataframe_info - Ratio/Relationship between CBG, TIF, and community
  metric                     - e.g) income, housing price, owner, ...
  Blockgroup_boundary_info   - Blockgroup boundary
  column_name_info           - the column name of metric e.g) ['Community', 'Less than $10,000', '$10,000 - $14,999', ... ]

  OUTPUT : metric data in Non TIF at the community level.
  -----------------------------------------------------------------------------------------------------------------------------------------
  test code
  test_incomeDistribution = pd.DataFrame({'geoid':['1500000US170313501001','1500000US170313501002','1500000US170313504001'],
                      'Less than $10,000':[[10,2,40],[2,4,8],[20,30,70]],
                      '$10,000 to $14,999':[[30,40,50],[4,6,10],[50,60,100]]})


  test_demogrpahicsDataFrame = pd.DataFrame({'Community':'DOUGLAS',
                                            'CBG in Community':[['1500000US170313501001','1500000US170313501002','1500000US170313504001']],
                                            'Non-TIF Ratio of CBG':[[0.1,0.5,0.1]]})
  column_name = ['Community','Less than $10,000','$10,000 to $14,999']
  aggregate_community_level_Non_TIF(test_demogrpahicsDataFrame,test_incomeDistribution,Blockgroup_boundary,column_name)

  result

  Community	Less than $10,000	$10,000 to $14,999
  DOUGLAS	[4.0, 5.2, 15.0]	[10.0, 13.0, 20.0]
  -----------------------------------------------------------------------------------------------------------------------------------------
  """
  GEOID = metric.columns[0]  # name of geoid e.g) 'geoid', 'GEO_ID', 'geo_id', 'GEOID'
  community_names = DemographicsDataframe_info['Community']
  no_data_of_geoid = []
  column_number = len(column_name_info)-1 # reason for -1 : 'Community'
  aggregated_Non_TIF = []
  for com_number, com_name in enumerate(community_names):
    lst = np.zeros((column_number,3))
    for range_name_index, range_name in enumerate(column_name_info[1:]):
      for GEOID_in_Community_index,GEOID_in_Community in enumerate(DemographicsDataframe_info['CBG in Community'].iloc[com_number]):
        if len(metric[metric[GEOID]==GEOID_in_Community][range_name])==0:
          no_data_of_geoid.append(GEOID_in_Community)
        else:
          ratio = DemographicsDataframe_info['Non-TIF Ratio of CBG'].iloc[com_number][GEOID_in_Community_index]
          lst[range_name_index] += np.array(metric[metric[GEOID]==GEOID_in_Community][range_name].iloc[0])*ratio

    aggregated_Non_TIF.append(np.array([[com_name]+list(lst)],dtype=object)[0])


  aggregated_Non_TIF_df = pd.DataFrame(aggregated_Non_TIF, columns = column_name_info)
  return aggregated_Non_TIF_df

# (Only median income)Calculate median income - Juhyun Jung

In [ ]:
def Calculate_Median_Income_from_Distribution(data):

  # Now, iterate for every race, every communities.

  communities = np.array(data['Community'])
  col_names = list(data.columns)
  col_names.pop(0)

  def cumulative_gamma_dist(x, shape, scale):
      return gamma.cdf(x, shape, scale=scale)

  def Coef_of_Determination(y_data, y_pred):

      y_mean = np.mean(y_data)

      # Calculate the total sum of squares (TSS)
      TSS = np.sum((y_data - y_mean) ** 2)

      # Calculate the residual sum of squares (RSS)
      RSS = np.sum((y_data - y_pred) ** 2)

      # Calculate the coefficient of determination (R^2)
      R_squared = 1 - (RSS / TSS)

      return R_squared

  median_incomes_list = []
  CoD_list = []

  ranges = [10000, 14999, 19999, 24999, 29999, 34999, 39999, 44999, 49999, 59999, 74999, 99999, 124999, 149999, 199999, np.inf]
  cf_ranges = (np.array(ranges[:-1]) - ranges[0]) / (ranges[-2] - ranges[0])


  for community in communities:
      comm_income = data[data['Community'] == community]


      # obtain percentage of population under the income in list 'ranges'
      W_list = []; B_list = []; L_list = []

      for col in col_names:
          W_list.append(comm_income[col].iloc[0][0])
          B_list.append(comm_income[col].iloc[0][1])
          L_list.append(comm_income[col].iloc[0][2])

      W_list = np.array(W_list); B_list = np.array(B_list); L_list = np.array(L_list)

      temp_CoD_list = []
      temp_median_income_list = []


      # there are communities with certain racial population = 0
      if np.sum(W_list) != 0:
          W_percentage = W_list/np.sum(W_list)
          W_cumulative_percentage = np.array([np.sum(W_percentage[:i+1]) for i in range(len(W_percentage))])

          pars, _ = curve_fit(f = cumulative_gamma_dist, xdata = cf_ranges, ydata = W_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_CoD_list.append(Coef_of_Determination(W_cumulative_percentage[:-1], cumulative_gamma_dist(cf_ranges, pars[0], pars[1])))
          temp_median_income_list.append(gamma.median(pars[0], scale=pars[1])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      if np.sum(B_list) != 0:
          B_percentage = B_list/np.sum(B_list)
          B_cumulative_percentage = np.array([np.sum(B_percentage[:i+1]) for i in range(len(B_percentage))])

          pars, _ = curve_fit(f = cumulative_gamma_dist, xdata = cf_ranges, ydata = B_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_CoD_list.append(Coef_of_Determination(B_cumulative_percentage[:-1], cumulative_gamma_dist(cf_ranges, pars[0], pars[1])))
          temp_median_income_list.append(gamma.median(pars[0], scale=pars[1])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      if np.sum(L_list) != 0:
          L_percentage = L_list/np.sum(L_list)
          L_cumulative_percentage = np.array([np.sum(L_percentage[:i+1]) for i in range(len(L_percentage))])

          pars, _ = curve_fit(f = cumulative_gamma_dist, xdata = cf_ranges, ydata = L_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_CoD_list.append(Coef_of_Determination(L_cumulative_percentage[:-1], cumulative_gamma_dist(cf_ranges, pars[0], pars[1])))
          temp_median_income_list.append(gamma.median(pars[0], scale=pars[1])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      median_incomes_list.append(temp_median_income_list)
      CoD_list.append(temp_CoD_list)

  # Mdeian Income Dataframe
  MI_df = pd.DataFrame(columns = ['Community', 'Median Income'])

  for idx, community in enumerate(communities):
      MI_df.loc[idx] = np.array([community, median_incomes_list[idx]], dtype = object)

  return MI_df

In [ ]:
def Calculate_Median_Income_from_Distribution(data):

  communities = np.array(data['Community'])
  col_names = list(data.columns)
  col_names.pop(0)

  def cumulative_exp_dist(x, lam):
      return 1 - np.exp(-lam * x)

  def Coef_of_Determination(y_data, y_pred):

      y_mean = np.mean(y_data)

      # Calculate the total sum of squares (TSS)
      TSS = np.sum((y_data - y_mean) ** 2)

      # Calculate the residual sum of squares (RSS)
      RSS = np.sum((y_data - y_pred) ** 2)

      # Calculate the coefficient of determination (R^2)
      R_squared = 1 - (RSS / TSS)

      return R_squared

  median_incomes_list = []
  CoD_list = []
  lambda_list = []


  ranges = [10000, 14999, 19999, 24999, 29999, 34999, 39999, 44999, 49999, 59999, 74999, 99999, 124999, 149999, 199999, np.inf]
  cf_ranges = (np.array(ranges[:-1]) - ranges[0]) / (ranges[-2] - ranges[0])


  for community in communities:
      comm_income = data[data['Community'] == community]


      # obtain percentage of population under the income in list 'ranges'
      W_list = []; B_list = []; L_list = []

      for col in col_names:
          W_list.append(comm_income[col].iloc[0][0])
          B_list.append(comm_income[col].iloc[0][1])
          L_list.append(comm_income[col].iloc[0][2])

      W_list = np.array(W_list); B_list = np.array(B_list); L_list = np.array(L_list)

      temp_lambda_list = []
      temp_CoD_list = []
      temp_median_income_list = []


      # there are communities with certain racial population = 0
      if np.sum(W_list) != 0:
          W_percentage = W_list/np.sum(W_list)
          W_cumulative_percentage = np.array([np.sum(W_percentage[:i+1]) for i in range(len(W_percentage))])

          pars, _ = curve_fit(f = cumulative_exp_dist, xdata = cf_ranges, ydata = W_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_lambda_list.append(pars[0])
          temp_CoD_list.append(Coef_of_Determination(W_cumulative_percentage[:-1], cumulative_exp_dist(cf_ranges, pars[0])))
          temp_median_income_list.append((np.log(2)/pars[0])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_lambda_list.append(0)
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      if np.sum(B_list) != 0:
          B_percentage = B_list/np.sum(B_list)
          B_cumulative_percentage = np.array([np.sum(B_percentage[:i+1]) for i in range(len(B_percentage))])

          pars, _ = curve_fit(f = cumulative_exp_dist, xdata = cf_ranges, ydata = B_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_lambda_list.append(pars[0])
          temp_CoD_list.append(Coef_of_Determination(B_cumulative_percentage[:-1], cumulative_exp_dist(cf_ranges, pars[0])))
          temp_median_income_list.append((np.log(2)/pars[0])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_lambda_list.append(0)
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      if np.sum(L_list) != 0:
          L_percentage = L_list/np.sum(L_list)
          L_cumulative_percentage = np.array([np.sum(L_percentage[:i+1]) for i in range(len(L_percentage))])

          pars, _ = curve_fit(f = cumulative_exp_dist, xdata = cf_ranges, ydata = L_cumulative_percentage[:-1], bounds = (-np.inf, np.inf))

          temp_lambda_list.append(pars[0])
          temp_CoD_list.append(Coef_of_Determination(L_cumulative_percentage[:-1], cumulative_exp_dist(cf_ranges, pars[0])))
          temp_median_income_list.append((np.log(2)/pars[0])*(ranges[-2] - ranges[0]) + ranges[0])
      else:
          temp_lambda_list.append(0)
          temp_CoD_list.append(0)
          temp_median_income_list.append(0)

      median_incomes_list.append(temp_median_income_list)
      CoD_list.append(temp_CoD_list)
      lambda_list.append(temp_lambda_list)

  # Mdeian Income Dataframe
  # MI_df = pd.DataFrame(columns = ['Community', 'Median Income', 'Coefficient of Determination', 'Lambda'])

  # Only Median Income
  MI_df = pd.DataFrame(columns = ['Community', 'Median Income'])

  for idx, community in enumerate(communities):
    MI_df.loc[idx] = np.array([community, median_incomes_list[idx]], dtype = object)

  return MI_df


# with open(path + 'median_income.pickle', 'wb') as file:
#     pickle.dump(MI_df, file)

# **3.** Seperate race - Juhyun Jung

In [ ]:
def seperate(aggregated_data_community):
  """
  aggregated_data_community : data is like [[10,20,30]]
  """
  total_number_of_community = len(aggregated_data_community)
  # print(aggregated_data_community.columns)
  metrics = aggregated_data_community.columns[-1]
  column_name = ['Community',f'{metrics} of White',f'{metrics} of Black',f'{metrics} of Latin']

  seperate_race_community_area_Median_income = pd.DataFrame(columns=column_name)
  arr = np.array(['name','White','Black','Latin'])

  for community_number in range(total_number_of_community):
    seperate_race_community_area_Median_income.loc[community_number] = np.array(arr)

  seperate_race_community_area_Median_income['Community'] = aggregated_data_community['Community']

  for community_number, community_name in enumerate(aggregated_data_community['Community']):
    for race_index, race in enumerate([f"{metrics} of White",f"{metrics} of Black",f"{metrics} of Latin"]):
        seperate_race_community_area_Median_income[race].iloc[community_number] = aggregated_data_community[metrics].iloc[community_number][race_index]

  return seperate_race_community_area_Median_income




# (Only Median Income) remove unnecessary characters - Juhyun Jung

In [ ]:
def preprocessing_for_median_income(seperate_race_community_area_Median_income_info):
  chars_to_remove = ['$', ',','Less than']
  # Apply the str.replace() method to each column
  for col in ['Median income of White', 'Median income of Black', 'Median income of Latin']:
      for char in chars_to_remove:
          seperate_race_community_area_Median_income_info[col] = seperate_race_community_area_Median_income_info[col].str.replace(char, '')
  chars_to_remove = ['to']
  # Apply the str.replace() method to each column
  for col in ['Median income of White', 'Median income of Black', 'Median income of Latin']:
      for char in chars_to_remove:
          seperate_race_community_area_Median_income_info[col] = seperate_race_community_area_Median_income_info[col].str.replace(char, ',')
  chars_to_remove = ['No data']
  # Apply the str.replace() method to each column
  for col in ['Median income of White', 'Median income of Black', 'Median income of Latin']:
      for char in chars_to_remove:
          seperate_race_community_area_Median_income_info[col] = seperate_race_community_area_Median_income_info[col].str.replace(char, '0')
  def calculate_average_for_median_income(cell_value):
    if ',' in cell_value:
        # Split the cell value at ','
        numbers = [int(num.strip()) for num in cell_value.split(',')]
        # Calculate the average
        return int(np.mean(numbers).round(-1))
    else:
        # Return the cell value as is if there is no comma
        return int(cell_value)
  # Apply the calculate_average function to each cell in the specified columns
  for col in ['Median income of White', 'Median income of Black', 'Median income of Latin']:
      seperate_race_community_area_Median_income_info[col] = seperate_race_community_area_Median_income_info[col].apply(calculate_average_for_median_income)
  return seperate_race_community_area_Median_income_info


# **4.** Save each year - Juhyun Jung, Kunwoo

In [ ]:
def aggregate_MI_df_list(df_list):

    list_median_income = []
    one_df = df_list[0]
    White_name = one_df.columns[1]
    Black_name = one_df.columns[2]
    Latin_name = one_df.columns[3]

    # print(one_df.columns)
    community_names = np.array(df_list[0]['Community'])
    columns = df_list[0].columns

    for name in community_names:
        W = [];B = [];L = []
        for df in df_list:
            W.append(df[df['Community'] == name][White_name].iloc[0])
            B.append(df[df['Community'] == name][Black_name].iloc[0])
            L.append(df[df['Community'] == name][Latin_name].iloc[0])

        list_median_income.append(np.array([name, W, B, L], dtype = object))


    return pd.DataFrame(list_median_income, columns = columns)

# **5.** Make matrix with many metric - Juhyun Jung

In [ ]:
def Make_matric_with_many_metrics(df_list):

    White = []
    Black = []
    Latin = []
    community_names = np.array(df_list[0]['Community'])
    columns_list = ['Community']
    count =0
    for metric_number in range(len(df_list)):
      metric_name = df_list[metric_number].columns[1]
      metric_name = metric_name.replace(' of ','')
      metric_name = metric_name.replace('White','')
      metric_name = metric_name.replace('Black','')
      metric_name = metric_name.replace('Latin','')
      columns_list.append(metric_name)
    # print(columns_list)

    for name in community_names:
      W = []
      B = []
      L = []
      for column_name_of_df, df in zip(columns_list[1:],df_list):
        W.append(df[df['Community'] == name][column_name_of_df+f' of White'].iloc[0])
        B.append(df[df['Community'] == name][column_name_of_df+f' of Black'].iloc[0])
        L.append(df[df['Community'] == name][column_name_of_df+f' of Latin'].iloc[0])
      White.append(np.array([[name]+W],dtype=object)[0])
      Black.append(np.array([[name]+B],dtype=object)[0])
      Latin.append(np.array([[name]+L],dtype=object)[0])

    aggregated_W = pd.DataFrame(White, columns = columns_list)
    aggregated_B = pd.DataFrame(Black, columns = columns_list)
    aggregated_L = pd.DataFrame(Latin, columns = columns_list)

    return aggregated_W,aggregated_B,aggregated_L


# **6.** Summary

In [ ]:
with open(path+'DemographicsDataframe.pickle', 'rb') as f:
	DemographicsDataframe = pickle.load(f)
with open(path+'racial_population.pickle', 'rb') as f:
	racial_population = pickle.load(f)

In [ ]:
print(len(DemographicsDataframe['CBG in Community'].iloc[0]))
print(len(DemographicsDataframe['Non-TIF Ratio of CBG'].iloc[0]))

20
20


# **6-1.** Median income

## It takes 11 mins
### Just use the save file


with open(path+'year_race_community_Median_TIF.pickle', 'rb') as f:
	year_race_community_Median_TIF = pickle.load(f)
with open(path+'year_race_community_Median_Non_TIF.pickle', 'rb') as f:
	year_race_community_Median_Non_TIF = pickle.load(f)

In [ ]:
column_name = ['Community','Less than $10,000', '$10,000 to $14,999', '$15,000 to $19,999',
        '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999',
        '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999',
        '$50,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
        '$100,000 to $124,999', '$125,000 to $149,999', '$150,000 to $199,999',
        '$200,000 or more']

#######################################################################

with open(path+'incomeDistribution2017.pickle', 'rb') as f:
  income_range_2017 = pickle.load(f)
income_range_2017.columns = income_range_2017.columns.str.replace('\\','')
community_area_income_range_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,income_range_2017,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2017 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2017)
seperate_race_community_area_Median_income_TIF_2017 = seperate(one_community_area_Median_income_TIF_2017)

community_area_income_range_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,income_range_2017,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2017 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2017)
seperate_race_community_area_Median_income_Non_TIF_2017 = seperate(one_community_area_Median_income_Non_TIF_2017)

#######################################################################

with open(path+'incomeDistribution2018.pickle', 'rb') as f:
  income_range_2018 = pickle.load(f)
income_range_2018.columns = income_range_2018.columns.str.replace('\\','')
community_area_income_range_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,income_range_2018,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2018 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2018)
seperate_race_community_area_Median_income_TIF_2018 = seperate(one_community_area_Median_income_TIF_2018)

community_area_income_range_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,income_range_2018,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2018 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2018)
seperate_race_community_area_Median_income_Non_TIF_2018 = seperate(one_community_area_Median_income_Non_TIF_2018)

#######################################################################

with open(path+'incomeDistribution2019.pickle', 'rb') as f:
  income_range_2019 = pickle.load(f)
income_range_2019.columns = income_range_2019.columns.str.replace('\\','')
community_area_income_range_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,income_range_2019,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2019 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2019)
seperate_race_community_area_Median_income_TIF_2019 = seperate(one_community_area_Median_income_TIF_2019)

community_area_income_range_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,income_range_2019,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2019 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2019)
seperate_race_community_area_Median_income_Non_TIF_2019 = seperate(one_community_area_Median_income_Non_TIF_2019)

#######################################################################

with open(path+'incomeDistribution2020.pickle', 'rb') as f:
  income_range_2020 = pickle.load(f)
income_range_2020.columns = income_range_2020.columns.str.replace('\\','')
community_area_income_range_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,income_range_2020,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2020 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2020)
seperate_race_community_area_Median_income_TIF_2020 = seperate(one_community_area_Median_income_TIF_2020)

community_area_income_range_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,income_range_2020,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2020 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2020)
seperate_race_community_area_Median_income_Non_TIF_2020 = seperate(one_community_area_Median_income_Non_TIF_2020)

#######################################################################


with open(path+'incomeDistribution2021.pickle', 'rb') as f:
  income_range_2021 = pickle.load(f)
income_range_2021.columns = income_range_2021.columns.str.replace('\\','')
community_area_income_range_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,income_range_2021,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2021 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2021)
seperate_race_community_area_Median_income_TIF_2021 = seperate(one_community_area_Median_income_TIF_2021)

community_area_income_range_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,income_range_2021,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2021 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2021)
seperate_race_community_area_Median_income_Non_TIF_2021 = seperate(one_community_area_Median_income_Non_TIF_2021)

#######################################################################

with open(path+'incomeDistribution2022.pickle', 'rb') as f:
  income_range_2022 = pickle.load(f)
income_range_2022.columns = income_range_2022.columns.str.replace('\\','')
community_area_income_range_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,income_range_2022,Blockgroup_boundary, column_name)
one_community_area_Median_income_TIF_2022 = Calculate_Median_Income_from_Distribution(community_area_income_range_TIF_2022)
seperate_race_community_area_Median_income_TIF_2022 = seperate(one_community_area_Median_income_TIF_2022)

community_area_income_range_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,income_range_2022,Blockgroup_boundary, column_name)
one_community_area_Median_income_Non_TIF_2022 = Calculate_Median_Income_from_Distribution(community_area_income_range_Non_TIF_2022)
seperate_race_community_area_Median_income_Non_TIF_2022 = seperate(one_community_area_Median_income_Non_TIF_2022)

#######################################################################

Median_TIF_list = [seperate_race_community_area_Median_income_TIF_2017,seperate_race_community_area_Median_income_TIF_2018,seperate_race_community_area_Median_income_TIF_2019,seperate_race_community_area_Median_income_TIF_2020,seperate_race_community_area_Median_income_TIF_2021,seperate_race_community_area_Median_income_TIF_2022]
Median_Non_TIF_list = [seperate_race_community_area_Median_income_Non_TIF_2017,seperate_race_community_area_Median_income_Non_TIF_2018,seperate_race_community_area_Median_income_Non_TIF_2019,seperate_race_community_area_Median_income_Non_TIF_2020,seperate_race_community_area_Median_income_Non_TIF_2021,seperate_race_community_area_Median_income_Non_TIF_2022]

year_race_community_Median_TIF  =  aggregate_MI_df_list(Median_TIF_list)
year_race_community_Median_Non_TIF  =  aggregate_MI_df_list(Median_Non_TIF_list)

with open(path + 'year_race_community_Median_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Median_TIF, file)
with open(path + 'year_race_community_Median_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Median_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_Median_TIF.pickle', 'rb') as f:
	year_race_community_Median_TIF = pickle.load(f)
with open(path+'year_race_community_Median_Non_TIF.pickle', 'rb') as f:
	year_race_community_Median_Non_TIF = pickle.load(f)

In [ ]:
year_race_community_Median_TIF.head(3)

,Community,Median Income of White,Median Income of Black,Median Income of Latin
0,DOUGLAS,"[35594.07590604172, 34469.69400794884, 36479.5...","[27290.32611366214, 27864.888886034856, 26992....","[35203.08066735219, 34213.02256655702, 34697.2..."
1,OAKLAND,"[33186.21309856826, 34860.508458423545, 36118....","[26736.779927666765, 26937.633704646894, 32818...","[28499.522413054954, 29310.33849052167, 33782...."
2,FULLER PARK,"[25970.69535552038, 26903.502142313133, 28251....","[22221.476402117143, 22102.050983581565, 21869...","[28877.699238498626, 26097.948737541446, 24741..."


In [ ]:
year_race_community_Median_Non_TIF.head(3)

,Community,Median Income of White,Median Income of Black,Median Income of Latin
0,DOUGLAS,"[39162.525716400276, 37583.111758587635, 40270...","[29433.358350515577, 27936.88693057242, 29142....","[32112.871694979272, 31548.083326099713, 32945..."
1,OAKLAND,"[28086.774228284827, 28018.54719573403, 28283....","[26767.496209164085, 27368.993758381108, 30443...","[30311.29975366348, 31956.52831407106, 31880.4..."
2,FULLER PARK,"[28020.32976581132, 29651.09455902325, 31813.8...","[23803.590629191767, 23866.736135247244, 23783...","[29111.637801220964, 27239.98976024836, 26137...."


# **6-2.** Ownership

## It takes 1 mins
## Just use the save file


with open(path+'year_race_community_Owner_TIF.pickle', 'rb') as f:
	year_race_community_Owner_TIF = pickle.load(f)
with open(path+'year_race_community_Owner_Non_TIF.pickle', 'rb') as f:
	year_race_community_Owner_Non_TIF = pickle.load(f)

In [ ]:
column_name = ['Community','Owner']

with open(path+'2017_owner.pickle', 'rb') as f:
  Owner_2017 = pickle.load(f)
one_community_area_Owner_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,Owner_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2017 = seperate(one_community_area_Owner_TIF_2017)

one_community_area_Owner_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,Owner_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2017 = seperate(one_community_area_Owner_Non_TIF_2017)

#######################################################################

with open(path+'2018_owner.pickle', 'rb') as f:
  Owner_2018 = pickle.load(f)
one_community_area_Owner_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,Owner_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2018 = seperate(one_community_area_Owner_TIF_2018)

one_community_area_Owner_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,Owner_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2018 = seperate(one_community_area_Owner_Non_TIF_2018)

#######################################################################

with open(path+'2019_owner.pickle', 'rb') as f:
  Owner_2019 = pickle.load(f)
one_community_area_Owner_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,Owner_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2019 = seperate(one_community_area_Owner_TIF_2019)

one_community_area_Owner_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,Owner_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2019 = seperate(one_community_area_Owner_Non_TIF_2019)

#######################################################################


with open(path+'2020_owner.pickle', 'rb') as f:
  Owner_2020 = pickle.load(f)
one_community_area_Owner_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,Owner_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2020 = seperate(one_community_area_Owner_TIF_2020)

one_community_area_Owner_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,Owner_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2020 = seperate(one_community_area_Owner_Non_TIF_2020)

#######################################################################


with open(path+'2021_owner.pickle', 'rb') as f:
  Owner_2021 = pickle.load(f)
one_community_area_Owner_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,Owner_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2021 = seperate(one_community_area_Owner_TIF_2021)

one_community_area_Owner_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,Owner_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2021 = seperate(one_community_area_Owner_Non_TIF_2021)

#######################################################################

with open(path+'2022_owner.pickle', 'rb') as f:
  Owner_2022 = pickle.load(f)
one_community_area_Owner_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,Owner_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_TIF_2022 = seperate(one_community_area_Owner_TIF_2022)

one_community_area_Owner_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,Owner_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Owner_Non_TIF_2022 = seperate(one_community_area_Owner_Non_TIF_2022)

#######################################################################

Owner_TIF_list = [seperate_race_community_area_Owner_TIF_2017,seperate_race_community_area_Owner_TIF_2018,seperate_race_community_area_Owner_TIF_2019,seperate_race_community_area_Owner_TIF_2020,seperate_race_community_area_Owner_TIF_2021,seperate_race_community_area_Owner_TIF_2022]
Owner_Non_TIF_list = [seperate_race_community_area_Owner_Non_TIF_2017,seperate_race_community_area_Owner_Non_TIF_2018,seperate_race_community_area_Owner_Non_TIF_2019,seperate_race_community_area_Owner_Non_TIF_2020,seperate_race_community_area_Owner_Non_TIF_2021,seperate_race_community_area_Owner_Non_TIF_2022]

year_race_community_Owner_TIF  =  aggregate_MI_df_list(Owner_TIF_list)
year_race_community_Owner_Non_TIF  =  aggregate_MI_df_list(Owner_Non_TIF_list)


with open(path + 'year_race_community_Owner_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Owner_TIF, file)
with open(path + 'year_race_community_Owner_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Owner_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_Owner_TIF.pickle', 'rb') as f:
	year_race_community_Owner_TIF = pickle.load(f)
with open(path+'year_race_community_Owner_Non_TIF.pickle', 'rb') as f:
	year_race_community_Owner_Non_TIF = pickle.load(f)

In [ ]:
year_race_community_Owner_TIF

,Community,Owner of White,Owner of Black,Owner of Latin
0,DOUGLAS,"[79.53138176830109, 75.94877726495861, 81.8344...","[672.9126467634773, 649.7152465485841, 631.577...","[24.947041526276426, 22.701759570799947, 24.17..."
1,OAKLAND,"[4.7951007318359995, 5.458196379993855, 5.7557...","[121.21816343512045, 133.33587871131735, 155.7...","[3.3821357957774123, 3.745056536087845, 4.0788..."
2,FULLER PARK,"[5.174790804711454, 5.058774727571219, 4.57797...","[245.04959612762102, 201.04268137747658, 176.7...","[13.287475627250322, 9.591566479853208, 7.6966..."
3,GRAND BOULEVARD,"[61.38699404648782, 66.19061532651638, 71.5178...","[2017.2668443656328, 2075.8828525285094, 2309....","[38.27519354264896, 41.62769903998119, 45.5643..."
4,KENWOOD,"[33.088985606150295, 35.316057680608296, 37.81...","[162.01113691158585, 182.4204597193802, 202.55...","[5.028757283254587, 5.459050689543513, 6.03368..."
...,...,...,...,...
72,MOUNT GREENWOOD,"[258.5824315602597, 299.26346657088135, 303.92...","[12.882944597424997, 13.944638230581328, 13.70...","[22.249367648329162, 25.674922538986486, 26.04..."
73,MORGAN PARK,"[568.8143554579917, 578.1020253372772, 602.466...","[2638.9568184726045, 2617.5829129692843, 2682....","[79.76233599927173, 80.18208226263715, 83.7566..."
74,OHARE,"[568.8143554579917, 578.1020253372772, 602.466...","[2638.9568184726045, 2617.5829129692843, 2682....","[79.76233599927173, 80.18208226263715, 83.7566..."
75,EDGEWATER,"[6163.501175240372, 5992.807312678456, 6255.12...","[1151.72210518949, 1202.2451261878887, 1217.07...","[1297.9096843050218, 1322.4939079298983, 1362...."


In [ ]:
year_race_community_Owner_Non_TIF.head(3)

,Community,Owner of White,Owner of Black,Owner of Latin
0,DOUGLAS,"[122.37269983080718, 120.08404364717919, 126.3...","[881.0976383342852, 859.9021694473948, 793.033...","[29.861599492915857, 29.372102836775365, 29.82..."
1,OAKLAND,"[14.38456982132064, 17.471455360150557, 16.832...","[321.5708133552753, 391.17606163908255, 397.53...","[6.833737574843362, 7.986571732610757, 8.13964..."
2,FULLER PARK,"[2.0281976747635504, 2.0925128205064376, 1.954...","[75.53617571991002, 66.1959320140998, 61.45359...","[3.525585088340658, 2.8332678535235245, 2.5336..."


# **6-3.** Renters

## It takes 1 mins
## Just use the save file


with open(path+'year_race_community_Renter_TIF.pickle', 'rb') as f:
	year_race_community_Renter_TIF = pickle.load(f)
with open(path+'year_race_community_Renter_Non_TIF.pickle', 'rb') as f:
	year_race_community_Renter_Non_TIF = pickle.load(f)

In [ ]:
column_name = ['Community','Renter']

with open(path+'2017_renter.pickle', 'rb') as f:
  Renter_2017 = pickle.load(f)
one_community_area_Renter_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,Renter_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2017 = seperate(one_community_area_Renter_TIF_2017)

one_community_area_Renter_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,Renter_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2017 = seperate(one_community_area_Renter_Non_TIF_2017)

#######################################################################

with open(path+'2018_renter.pickle', 'rb') as f:
  Renter_2018 = pickle.load(f)
one_community_area_Renter_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,Renter_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2018 = seperate(one_community_area_Renter_TIF_2018)

one_community_area_Renter_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,Renter_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2018 = seperate(one_community_area_Renter_Non_TIF_2018)

#######################################################################

with open(path+'2019_renter.pickle', 'rb') as f:
  Renter_2019 = pickle.load(f)
one_community_area_Renter_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,Renter_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2019 = seperate(one_community_area_Renter_TIF_2019)

one_community_area_Renter_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,Renter_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2019 = seperate(one_community_area_Renter_Non_TIF_2019)

#######################################################################


with open(path+'2020_renter.pickle', 'rb') as f:
  Renter_2020 = pickle.load(f)
one_community_area_Renter_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,Renter_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2020 = seperate(one_community_area_Renter_TIF_2020)

one_community_area_Renter_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,Renter_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2020 = seperate(one_community_area_Renter_Non_TIF_2020)

#######################################################################


with open(path+'2021_renter.pickle', 'rb') as f:
  Renter_2021 = pickle.load(f)
one_community_area_Renter_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,Renter_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2021 = seperate(one_community_area_Renter_TIF_2021)

one_community_area_Renter_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,Renter_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2021 = seperate(one_community_area_Renter_Non_TIF_2021)

#######################################################################

with open(path+'2022_renter.pickle', 'rb') as f:
  Renter_2022 = pickle.load(f)
one_community_area_Renter_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,Renter_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_TIF_2022 = seperate(one_community_area_Renter_TIF_2022)

one_community_area_Renter_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,Renter_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Renter_Non_TIF_2022 = seperate(one_community_area_Renter_Non_TIF_2022)

#######################################################################

Renter_TIF_list = [seperate_race_community_area_Renter_TIF_2017,seperate_race_community_area_Renter_TIF_2018,seperate_race_community_area_Renter_TIF_2019,seperate_race_community_area_Renter_TIF_2020,seperate_race_community_area_Renter_TIF_2021,seperate_race_community_area_Renter_TIF_2022]
Renter_Non_TIF_list = [seperate_race_community_area_Renter_Non_TIF_2017,seperate_race_community_area_Renter_Non_TIF_2018,seperate_race_community_area_Renter_Non_TIF_2019,seperate_race_community_area_Renter_Non_TIF_2020,seperate_race_community_area_Renter_Non_TIF_2021,seperate_race_community_area_Renter_Non_TIF_2022]

year_race_community_Renter_TIF  =  aggregate_MI_df_list(Renter_TIF_list)
year_race_community_Renter_Non_TIF  =  aggregate_MI_df_list(Renter_Non_TIF_list)

with open(path + 'year_race_community_Renter_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Renter_TIF, file)
with open(path + 'year_race_community_Renter_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Renter_Non_TIF, file)




In [ ]:
with open(path+'year_race_community_Renter_TIF.pickle', 'rb') as f:
	year_race_community_Renter_TIF = pickle.load(f)
with open(path+'year_race_community_Renter_Non_TIF.pickle', 'rb') as f:
	year_race_community_Renter_Non_TIF = pickle.load(f)

In [ ]:
year_race_community_Renter_TIF.head(3)

,Community,Renter of White,Renter of Black,Renter of Latin
0,DOUGLAS,"[192.92747840074895, 207.33559817408704, 215.5...","[2572.329928436238, 2650.5891356672496, 2740.7...","[62.79535110414741, 64.15795485122725, 66.4285..."
1,OAKLAND,"[10.938704029399245, 11.123654292052986, 11.70...","[615.3257824712373, 630.0643895922321, 654.987...","[10.758041423002984, 11.037081021483285, 11.53..."
2,FULLER PARK,"[8.960021025397037, 9.293122480715713, 9.51689...","[516.2656208085832, 579.9585834732147, 651.045...","[21.119475746675743, 26.83022348685531, 31.897..."


In [ ]:
year_race_community_Renter_Non_TIF.head(3)

,Community,Renter of White,Renter of Black,Renter of Latin
0,DOUGLAS,"[346.74385729045525, 342.68341791127, 351.0104...","[4269.725065638216, 4263.699343458592, 4349.93...","[119.85478635791048, 118.96518259261276, 120.4..."
1,OAKLAND,"[43.577663325065394, 41.26417487100986, 43.730...","[1588.63513948589, 1618.8028178233244, 1684.84...","[18.71714383046386, 19.716680358296966, 20.566..."
2,FULLER PARK,"[3.22051846000219, 3.1715852297963636, 3.05234...","[140.7483305612302, 152.8080758327227, 165.350...","[5.230685624713474, 6.2030112866911695, 7.0035..."


# **6.4** Employment

In [ ]:
column_name = ['Community','Employment']

with open(path+'2017_employed.pickle', 'rb') as f:
  Employment_2017 = pickle.load(f)
one_community_area_Employment_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,Employment_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2017 = seperate(one_community_area_Employment_TIF_2017)

one_community_area_Employment_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,Employment_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2017 = seperate(one_community_area_Employment_Non_TIF_2017)

#######################################################################

with open(path+'2018_employed.pickle', 'rb') as f:
  Employment_2018 = pickle.load(f)
one_community_area_Employment_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,Employment_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2018 = seperate(one_community_area_Employment_TIF_2018)

one_community_area_Employment_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,Employment_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2018 = seperate(one_community_area_Employment_Non_TIF_2018)

#######################################################################

with open(path+'2019_employed.pickle', 'rb') as f:
  Employment_2019 = pickle.load(f)
one_community_area_Employment_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,Employment_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2019 = seperate(one_community_area_Employment_TIF_2019)

one_community_area_Employment_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,Employment_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2019 = seperate(one_community_area_Employment_Non_TIF_2019)

#######################################################################


with open(path+'2020_employed.pickle', 'rb') as f:
  Employment_2020 = pickle.load(f)
one_community_area_Employment_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,Employment_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2020 = seperate(one_community_area_Employment_TIF_2020)

one_community_area_Employment_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,Employment_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2020 = seperate(one_community_area_Employment_Non_TIF_2020)

#######################################################################


with open(path+'2021_employed.pickle', 'rb') as f:
  Employment_2021 = pickle.load(f)
one_community_area_Employment_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,Employment_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2021 = seperate(one_community_area_Employment_TIF_2021)

one_community_area_Employment_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,Employment_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2021 = seperate(one_community_area_Employment_Non_TIF_2021)

#######################################################################

with open(path+'2022_employed.pickle', 'rb') as f:
  Employment_2022 = pickle.load(f)
one_community_area_Employment_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,Employment_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_TIF_2022 = seperate(one_community_area_Employment_TIF_2022)

one_community_area_Employment_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,Employment_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Employment_Non_TIF_2022 = seperate(one_community_area_Employment_Non_TIF_2022)

#######################################################################

Employment_TIF_list = [seperate_race_community_area_Employment_TIF_2017,seperate_race_community_area_Employment_TIF_2018,seperate_race_community_area_Employment_TIF_2019,seperate_race_community_area_Employment_TIF_2020,seperate_race_community_area_Employment_TIF_2021,seperate_race_community_area_Employment_TIF_2022]
Employment_Non_TIF_list = [seperate_race_community_area_Employment_Non_TIF_2017,seperate_race_community_area_Employment_Non_TIF_2018,seperate_race_community_area_Employment_Non_TIF_2019,seperate_race_community_area_Employment_Non_TIF_2020,seperate_race_community_area_Employment_Non_TIF_2021,seperate_race_community_area_Employment_Non_TIF_2022]

year_race_community_Employment_TIF  =  aggregate_MI_df_list(Employment_TIF_list)
year_race_community_Employment_Non_TIF  =  aggregate_MI_df_list(Employment_Non_TIF_list)

with open(path + 'year_race_community_Employment_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Employment_TIF, file)
with open(path + 'year_race_community_Employment_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Employment_Non_TIF, file)




In [ ]:
with open(path+'year_race_community_Employment_TIF.pickle', 'rb') as f:
	year_race_community_Employment_TIF = pickle.load(f)
with open(path+'year_race_community_Employment_Non_TIF.pickle', 'rb') as f:
	year_race_community_Employment_Non_TIF = pickle.load(f)
year_race_community_Employment_TIF.head(3)

In [ ]:
year_race_community_Employment_Non_TIF.head(3)

,Community,Employment of White,Employment of Black,Employment of Latin
0,DOUGLAS,"[610.178716494088, 601.7935718593101, 622.0138...","[4838.108781410493, 4819.398746735623, 4806.44...","[159.8571165280962, 158.44084809454077, 161.40..."
1,OAKLAND,"[65.56061502634356, 64.84094867216544, 66.0646...","[2116.165509034402, 2230.4667091577585, 2333.5...","[28.144265165927077, 32.1246054341931, 33.5225..."
2,FULLER PARK,"[5.784216730809196, 5.568225129181762, 5.55390...","[190.69318689254462, 180.88627103674855, 184.6...","[9.148248133070485, 8.521166074692042, 8.07342..."


# **6.5** Unemployment

In [ ]:
column_name = ['Community','Unemployment']

with open(path+'2017_unemployed.pickle', 'rb') as f:
  Unemployment_2017 = pickle.load(f)
one_community_area_Unemployment_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,Unemployment_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2017 = seperate(one_community_area_Unemployment_TIF_2017)

one_community_area_Unemployment_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,Unemployment_2017,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2017 = seperate(one_community_area_Unemployment_Non_TIF_2017)

#######################################################################

with open(path+'2018_unemployed.pickle', 'rb') as f:
  Unemployment_2018 = pickle.load(f)
one_community_area_Unemployment_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,Unemployment_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2018 = seperate(one_community_area_Unemployment_TIF_2018)

one_community_area_Unemployment_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,Unemployment_2018,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2018 = seperate(one_community_area_Unemployment_Non_TIF_2018)

#######################################################################

with open(path+'2019_unemployed.pickle', 'rb') as f:
  Unemployment_2019 = pickle.load(f)
one_community_area_Unemployment_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,Unemployment_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2019 = seperate(one_community_area_Unemployment_TIF_2019)

one_community_area_Unemployment_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,Unemployment_2019,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2019 = seperate(one_community_area_Unemployment_Non_TIF_2019)

#######################################################################


with open(path+'2020_unemployed.pickle', 'rb') as f:
  Unemployment_2020 = pickle.load(f)
one_community_area_Unemployment_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,Unemployment_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2020 = seperate(one_community_area_Unemployment_TIF_2020)

one_community_area_Unemployment_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,Unemployment_2020,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2020 = seperate(one_community_area_Unemployment_Non_TIF_2020)

#######################################################################


with open(path+'2021_unemployed.pickle', 'rb') as f:
  Unemployment_2021 = pickle.load(f)
one_community_area_Unemployment_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,Unemployment_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2021 = seperate(one_community_area_Unemployment_TIF_2021)

one_community_area_Unemployment_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,Unemployment_2021,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2021 = seperate(one_community_area_Unemployment_Non_TIF_2021)

#######################################################################

with open(path+'2022_unemployed.pickle', 'rb') as f:
  Unemployment_2022 = pickle.load(f)
one_community_area_Unemployment_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,Unemployment_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_TIF_2022 = seperate(one_community_area_Unemployment_TIF_2022)

one_community_area_Unemployment_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,Unemployment_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_Unemployment_Non_TIF_2022 = seperate(one_community_area_Unemployment_Non_TIF_2022)

#######################################################################

Unemployment_TIF_list = [seperate_race_community_area_Unemployment_TIF_2017,seperate_race_community_area_Unemployment_TIF_2018,seperate_race_community_area_Unemployment_TIF_2019,seperate_race_community_area_Unemployment_TIF_2020,seperate_race_community_area_Unemployment_TIF_2021,seperate_race_community_area_Unemployment_TIF_2022]
Unemployment_Non_TIF_list = [seperate_race_community_area_Unemployment_Non_TIF_2017,seperate_race_community_area_Unemployment_Non_TIF_2018,seperate_race_community_area_Unemployment_Non_TIF_2019,seperate_race_community_area_Unemployment_Non_TIF_2020,seperate_race_community_area_Unemployment_Non_TIF_2021,seperate_race_community_area_Unemployment_Non_TIF_2022]

year_race_community_Unemployment_TIF  =  aggregate_MI_df_list(Unemployment_TIF_list)
year_race_community_Unemployment_Non_TIF  =  aggregate_MI_df_list(Unemployment_Non_TIF_list)

with open(path + 'year_race_community_Unemployment_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Unemployment_TIF, file)
with open(path + 'year_race_community_Unemployment_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_Unemployment_Non_TIF, file)




In [ ]:
with open(path+'year_race_community_Unemployment_TIF.pickle', 'rb') as f:
	year_race_community_Unemployment_TIF = pickle.load(f)
with open(path+'year_race_community_Unemployment_Non_TIF.pickle', 'rb') as f:
	year_race_community_Unemployment_Non_TIF = pickle.load(f)
year_race_community_Unemployment_TIF.head(3)

,Community,Unemployment of White,Unemployment of Black,Unemployment of Latin
0,DOUGLAS,"[658.2655179655825, 655.6931929831686, 656.190...","[2492.219335144357, 2481.0463504956656, 2623.7...","[120.24509943929466, 117.87143848139013, 122.3..."
1,OAKLAND,"[8.144040594793669, 7.680660256975781, 7.93125...","[426.6115082347286, 411.92396028052474, 394.80...","[7.687130230796329, 7.451846305922344, 7.40766..."
2,FULLER PARK,"[15.216455150429509, 15.933041931519332, 12.88...","[823.8671062239946, 839.5409216324596, 793.950...","[40.79788680442632, 42.816637269064614, 43.228..."


In [ ]:
year_race_community_Unemployment_Non_TIF.head(3)

,Community,Unemployment of White,Unemployment of Black,Unemployment of Latin
0,DOUGLAS,"[738.6985733770254, 760.8551798950898, 742.258...","[3860.853240984495, 4040.795585874993, 3801.82...","[156.4350126447425, 163.1559932913808, 156.427..."
1,OAKLAND,"[33.00316245447184, 27.132184324982966, 30.065...","[1094.8939750355792, 959.1251551210538, 1001.8...","[13.087804924206825, 12.527063616021191, 12.83..."
2,FULLER PARK,"[5.554726302738618, 5.765332124563243, 4.31480...","[230.57585135115127, 230.80372566198236, 207.3...","[9.941973933676849, 10.089099347877365, 9.5785..."


# **6.6** Education A

In [ ]:
column_name = ['Community','No schooling completed']

with open(path+'educationDistributionA2016.pickle', 'rb') as f:
  EducationDistributionA2016 = pickle.load(f)
one_community_area_EducationA_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionA2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2016 = seperate(one_community_area_EducationA_TIF_2016)

one_community_area_EducationA_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionA2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2016 = seperate(one_community_area_EducationA_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionA2017.pickle', 'rb') as f:
  EducationDistributionA2017 = pickle.load(f)
one_community_area_EducationA_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionA2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2017 = seperate(one_community_area_EducationA_TIF_2017)

one_community_area_EducationA_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionA2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2017 = seperate(one_community_area_EducationA_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionA2018.pickle', 'rb') as f:
  EducationDistributionA2018 = pickle.load(f)
one_community_area_EducationA_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionA2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2018 = seperate(one_community_area_EducationA_TIF_2018)

one_community_area_EducationA_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionA2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2018 = seperate(one_community_area_EducationA_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionA2019.pickle', 'rb') as f:
  EducationDistributionA2019 = pickle.load(f)
one_community_area_EducationA_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionA2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2019 = seperate(one_community_area_EducationA_TIF_2019)

one_community_area_EducationA_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionA2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2019 = seperate(one_community_area_EducationA_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionA2020.pickle', 'rb') as f:
  EducationDistributionA2020 = pickle.load(f)
one_community_area_EducationA_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionA2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2020 = seperate(one_community_area_EducationA_TIF_2020)

one_community_area_EducationA_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionA2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2020 = seperate(one_community_area_EducationA_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionA2021.pickle', 'rb') as f:
  EducationDistributionA2021 = pickle.load(f)
one_community_area_EducationA_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionA2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2021 = seperate(one_community_area_EducationA_TIF_2021)

one_community_area_EducationA_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionA2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2021 = seperate(one_community_area_EducationA_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionA2022.pickle', 'rb') as f:
  EducationDistributionA2022 = pickle.load(f)
one_community_area_EducationA_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionA2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_TIF_2022 = seperate(one_community_area_EducationA_TIF_2022)

one_community_area_EducationA_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionA2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationA_Non_TIF_2022 = seperate(one_community_area_EducationA_Non_TIF_2022)

#######################################################################

EducationA_TIF_list = [seperate_race_community_area_EducationA_TIF_2017,seperate_race_community_area_EducationA_TIF_2018,seperate_race_community_area_EducationA_TIF_2019,seperate_race_community_area_EducationA_TIF_2020,seperate_race_community_area_EducationA_TIF_2021,seperate_race_community_area_EducationA_TIF_2022]
EducationA_Non_TIF_list = [seperate_race_community_area_EducationA_Non_TIF_2017,seperate_race_community_area_EducationA_Non_TIF_2018,seperate_race_community_area_EducationA_Non_TIF_2019,seperate_race_community_area_EducationA_Non_TIF_2020,seperate_race_community_area_EducationA_Non_TIF_2021,seperate_race_community_area_EducationA_Non_TIF_2022]

year_race_community_EducationA_TIF  =  aggregate_MI_df_list(EducationA_TIF_list)
year_race_community_EducationA_Non_TIF  =  aggregate_MI_df_list(EducationA_Non_TIF_list)

with open(path + 'year_race_community_EducationA_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationA_TIF, file)
with open(path + 'year_race_community_EducationA_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationA_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationA_TIF.pickle', 'rb') as f:
	year_race_community_EducationA_TIF = pickle.load(f)
with open(path+'year_race_community_EducationA_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationA_Non_TIF = pickle.load(f)
year_race_community_EducationA_TIF.head(3)

,Community,No schooling completed of White,No schooling completed of Black,No schooling completed of Latin
0,DOUGLAS,"[1.5488410749886246, 2.2364141708246823, 1.555...","[32.52290948166825, 43.91717691773281, 34.0698...","[0.6683532184508153, 0.7812906706498434, 0.670..."
1,OAKLAND,"[0.23732364641642828, 0.2165328420111768, 0.40...","[22.939056270861784, 20.922348243552396, 28.46...","[0.4712582331857, 0.42967662437519705, 0.62532..."
2,FULLER PARK,"[0.8484010000617457, 0.5800634847427503, 0.327...","[42.18456066870396, 27.980166770290502, 15.385...","[1.0334853208970336, 0.7651478055780382, 0.513..."


In [ ]:
year_race_community_EducationA_Non_TIF.head(3)

,Community,No schooling completed of White,No schooling completed of Black,No schooling completed of Latin
0,DOUGLAS,"[2.56111445301429, 6.083558828744017, 5.464113...","[44.881796469220184, 89.14252423498606, 70.469...","[0.8015528554351884, 1.7884831524251799, 1.828..."
1,OAKLAND,"[0.22057421669007254, 0.21154522966504347, 0.5...","[15.719905559429375, 14.844093818001554, 26.20...","[0.2046570392339926, 0.1865990651839344, 0.410..."
2,FULLER PARK,"[0.336446624255049, 0.23652396657133684, 0.140...","[13.180595947553787, 9.107741832536036, 5.3905...","[0.40015749076871726, 0.3002348330850051, 0.20..."


# **6.7** Education B

In [ ]:
column_name = ['Community','Less than High School']

with open(path+'educationDistributionB2016.pickle', 'rb') as f:
  EducationDistributionB2016 = pickle.load(f)
one_community_area_EducationB_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionB2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2016 = seperate(one_community_area_EducationB_TIF_2016)

one_community_area_EducationB_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionB2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2016 = seperate(one_community_area_EducationB_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionB2017.pickle', 'rb') as f:
  EducationDistributionB2017 = pickle.load(f)
one_community_area_EducationB_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionB2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2017 = seperate(one_community_area_EducationB_TIF_2017)

one_community_area_EducationB_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionB2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2017 = seperate(one_community_area_EducationB_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionB2018.pickle', 'rb') as f:
  EducationDistributionB2018 = pickle.load(f)
one_community_area_EducationB_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionB2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2018 = seperate(one_community_area_EducationB_TIF_2018)

one_community_area_EducationB_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionB2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2018 = seperate(one_community_area_EducationB_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionB2019.pickle', 'rb') as f:
  EducationDistributionB2019 = pickle.load(f)
one_community_area_EducationB_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionB2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2019 = seperate(one_community_area_EducationB_TIF_2019)

one_community_area_EducationB_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionB2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2019 = seperate(one_community_area_EducationB_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionB2020.pickle', 'rb') as f:
  EducationDistributionB2020 = pickle.load(f)
one_community_area_EducationB_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionB2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2020 = seperate(one_community_area_EducationB_TIF_2020)

one_community_area_EducationB_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionB2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2020 = seperate(one_community_area_EducationB_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionB2021.pickle', 'rb') as f:
  EducationDistributionB2021 = pickle.load(f)
one_community_area_EducationB_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionB2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2021 = seperate(one_community_area_EducationB_TIF_2021)

one_community_area_EducationB_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionB2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2021 = seperate(one_community_area_EducationB_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionB2022.pickle', 'rb') as f:
  EducationDistributionB_2022 = pickle.load(f)
one_community_area_EducationB_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionB_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_TIF_2022 = seperate(one_community_area_EducationB_TIF_2022)

one_community_area_EducationB_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionB_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationB_Non_TIF_2022 = seperate(one_community_area_EducationB_Non_TIF_2022)

#######################################################################

EducationB_TIF_list = [seperate_race_community_area_EducationB_TIF_2017,seperate_race_community_area_EducationB_TIF_2018,seperate_race_community_area_EducationB_TIF_2019,seperate_race_community_area_EducationB_TIF_2020,seperate_race_community_area_EducationB_TIF_2021,seperate_race_community_area_EducationB_TIF_2022]
EducationB_Non_TIF_list = [seperate_race_community_area_EducationB_Non_TIF_2017,seperate_race_community_area_EducationB_Non_TIF_2018,seperate_race_community_area_EducationB_Non_TIF_2019,seperate_race_community_area_EducationB_Non_TIF_2020,seperate_race_community_area_EducationB_Non_TIF_2021,seperate_race_community_area_EducationB_Non_TIF_2022]

year_race_community_EducationB_TIF  =  aggregate_MI_df_list(EducationB_TIF_list)
year_race_community_EducationB_Non_TIF  =  aggregate_MI_df_list(EducationB_Non_TIF_list)

with open(path + 'year_race_community_EducationB_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationB_TIF, file)
with open(path + 'year_race_community_EducationB_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationB_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationB_TIF.pickle', 'rb') as f:
	year_race_community_EducationB_TIF = pickle.load(f)
with open(path+'year_race_community_EducationB_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationB_Non_TIF = pickle.load(f)
year_race_community_EducationB_TIF.head(3)

,Community,Less than High School of White,Less than High School of Black,Less than High School of Latin
0,DOUGLAS,"[15.932794418798064, 16.33607555133391, 12.383...","[105.27132990689827, 95.75773067692778, 82.229...","[3.6152884604339066, 3.1103156225370365, 2.513..."
1,OAKLAND,"[1.126313743408786, 0.32984793318591565, 0.229...","[49.12697395523004, 11.4472779330307, 8.642960...","[1.1216987873600406, 0.22251901970686425, 0.19..."
2,FULLER PARK,"[1.1039986760881735, 1.2698510010161874, 0.837...","[55.01165554556346, 52.187508508934194, 40.511...","[3.247685453079304, 4.403614930865536, 3.73585..."


In [ ]:
year_race_community_EducationB_Non_TIF.head(3)

,Community,Less than High School of White,Less than High School of Black,Less than High School of Latin
0,DOUGLAS,"[19.738222589162184, 22.364990546198413, 17.72...","[144.14119104793502, 140.19805228443732, 117.7...","[4.807716993776683, 4.80310932912329, 3.971254..."
1,OAKLAND,"[2.6248040428504695, 1.5910047487912, 0.917654...","[82.09077113262859, 49.56914749325575, 37.1884...","[1.4772929236797594, 0.5968207810231503, 0.437..."
2,FULLER PARK,"[0.4105995081146546, 0.4849443781408983, 0.289...","[15.353187843347184, 14.522038017954577, 9.778...","[0.7613828735528044, 0.9442603764200688, 0.714..."


# **6.8** Education C

In [ ]:
column_name = ['Community','High School, No Diploma']

with open(path+'educationDistributionC2016.pickle', 'rb') as f:
  EducationDistributionC2016 = pickle.load(f)
one_community_area_EducationC_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionC2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2016 = seperate(one_community_area_EducationC_TIF_2016)

one_community_area_EducationC_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionC2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2016 = seperate(one_community_area_EducationC_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionC2017.pickle', 'rb') as f:
  EducationDistributionC2017 = pickle.load(f)
one_community_area_EducationC_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionC2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2017 = seperate(one_community_area_EducationC_TIF_2017)

one_community_area_EducationC_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionC2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2017 = seperate(one_community_area_EducationC_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionC2018.pickle', 'rb') as f:
  EducationDistributionC2018 = pickle.load(f)
one_community_area_EducationC_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionC2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2018 = seperate(one_community_area_EducationC_TIF_2018)

one_community_area_EducationC_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionC2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2018 = seperate(one_community_area_EducationC_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionC2019.pickle', 'rb') as f:
  EducationDistributionC2019 = pickle.load(f)
one_community_area_EducationC_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionC2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2019 = seperate(one_community_area_EducationC_TIF_2019)

one_community_area_EducationC_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionC2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2019 = seperate(one_community_area_EducationC_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionC2020.pickle', 'rb') as f:
  EducationDistributionC2020 = pickle.load(f)
one_community_area_EducationC_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionC2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2020 = seperate(one_community_area_EducationC_TIF_2020)

one_community_area_EducationC_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionC2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2020 = seperate(one_community_area_EducationC_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionC2021.pickle', 'rb') as f:
  EducationDistributionC2021 = pickle.load(f)
one_community_area_EducationC_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionC2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2021 = seperate(one_community_area_EducationC_TIF_2021)

one_community_area_EducationC_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionC2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2021 = seperate(one_community_area_EducationC_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionC2022.pickle', 'rb') as f:
  EducationDistributionC_2022 = pickle.load(f)
one_community_area_EducationC_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionC_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_TIF_2022 = seperate(one_community_area_EducationC_TIF_2022)

one_community_area_EducationC_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionC_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationC_Non_TIF_2022 = seperate(one_community_area_EducationC_Non_TIF_2022)

#######################################################################

EducationC_TIF_list = [seperate_race_community_area_EducationC_TIF_2017,seperate_race_community_area_EducationC_TIF_2018,seperate_race_community_area_EducationC_TIF_2019,seperate_race_community_area_EducationC_TIF_2020,seperate_race_community_area_EducationC_TIF_2021,seperate_race_community_area_EducationC_TIF_2022]
EducationC_Non_TIF_list = [seperate_race_community_area_EducationC_Non_TIF_2017,seperate_race_community_area_EducationC_Non_TIF_2018,seperate_race_community_area_EducationC_Non_TIF_2019,seperate_race_community_area_EducationC_Non_TIF_2020,seperate_race_community_area_EducationC_Non_TIF_2021,seperate_race_community_area_EducationC_Non_TIF_2022]

year_race_community_EducationC_TIF  =  aggregate_MI_df_list(EducationC_TIF_list)
year_race_community_EducationC_Non_TIF  =  aggregate_MI_df_list(EducationC_Non_TIF_list)

with open(path + 'year_race_community_EducationC_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationC_TIF, file)
with open(path + 'year_race_community_EducationC_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationC_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationC_TIF.pickle', 'rb') as f:
	year_race_community_EducationC_TIF = pickle.load(f)
with open(path+'year_race_community_EducationC_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationC_Non_TIF = pickle.load(f)
year_race_community_EducationC_TIF.head(3)

,Community,"High School, No Diploma of White","High School, No Diploma of Black","High School, No Diploma of Latin"
0,DOUGLAS,"[41.19837078813596, 39.62212145267087, 50.9782...","[450.15034292358433, 381.65098946652546, 416.1...","[11.866799467639355, 10.500556983964058, 12.14..."
1,OAKLAND,"[1.8275303953506716, 1.4746272985407898, 0.813...","[111.04798008733147, 99.44050367742462, 44.728...","[2.1530113033701244, 1.9382338892450666, 0.770..."
2,FULLER PARK,"[5.382440489821066, 5.072366330904897, 3.32520...","[240.92025411728602, 224.53066317726498, 216.2...","[10.823423667936241, 8.963709023743869, 8.8754..."


In [ ]:
year_race_community_EducationC_Non_TIF.head(3)

,Community,"High School, No Diploma of White","High School, No Diploma of Black","High School, No Diploma of Latin"
0,DOUGLAS,"[57.41838939631556, 59.500272442780314, 64.985...","[631.5145988055727, 683.8413453179967, 597.867...","[18.987589230946003, 20.482607362466126, 20.11..."
1,OAKLAND,"[6.253127310486004, 5.967476101715717, 5.61341...","[263.4390040311122, 237.40261290426866, 220.47...","[2.5103234812000212, 2.3849963208043303, 1.782..."
2,FULLER PARK,"[2.139186891073504, 2.020080196380714, 1.15542...","[74.24843584621972, 69.45343666680228, 58.9205...","[3.083004354546932, 2.6782338741206053, 2.2762..."


# **6.9** Education D

In [ ]:
column_name = ['Community','High School/GED or alternative credential']

with open(path+'educationDistributionD2016.pickle', 'rb') as f:
  EducationDistributionD2016 = pickle.load(f)
one_community_area_EducationD_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionD2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2016 = seperate(one_community_area_EducationD_TIF_2016)

one_community_area_EducationD_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionD2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2016 = seperate(one_community_area_EducationD_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionD2017.pickle', 'rb') as f:
  EducationDistributionD2017 = pickle.load(f)
one_community_area_EducationD_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionD2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2017 = seperate(one_community_area_EducationD_TIF_2017)

one_community_area_EducationD_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionD2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2017 = seperate(one_community_area_EducationD_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionD2018.pickle', 'rb') as f:
  EducationDistributionD2018 = pickle.load(f)
one_community_area_EducationD_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionD2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2018 = seperate(one_community_area_EducationD_TIF_2018)

one_community_area_EducationD_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionD2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2018 = seperate(one_community_area_EducationD_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionD2019.pickle', 'rb') as f:
  EducationDistributionD2019 = pickle.load(f)
one_community_area_EducationD_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionD2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2019 = seperate(one_community_area_EducationD_TIF_2019)

one_community_area_EducationD_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionD2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2019 = seperate(one_community_area_EducationD_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionD2020.pickle', 'rb') as f:
  EducationDistributionD2020 = pickle.load(f)
one_community_area_EducationD_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionD2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2020 = seperate(one_community_area_EducationD_TIF_2020)

one_community_area_EducationD_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionD2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2020 = seperate(one_community_area_EducationD_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionD2021.pickle', 'rb') as f:
  EducationDistributionD2021 = pickle.load(f)
one_community_area_EducationD_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionD2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2021 = seperate(one_community_area_EducationD_TIF_2021)

one_community_area_EducationD_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionD2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2021 = seperate(one_community_area_EducationD_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionD2022.pickle', 'rb') as f:
  EducationDistributionD_2022 = pickle.load(f)
one_community_area_EducationD_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionD_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_TIF_2022 = seperate(one_community_area_EducationD_TIF_2022)

one_community_area_EducationD_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionD_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationD_Non_TIF_2022 = seperate(one_community_area_EducationD_Non_TIF_2022)

#######################################################################

EducationD_TIF_list = [seperate_race_community_area_EducationD_TIF_2017,seperate_race_community_area_EducationD_TIF_2018,seperate_race_community_area_EducationD_TIF_2019,seperate_race_community_area_EducationD_TIF_2020,seperate_race_community_area_EducationD_TIF_2021,seperate_race_community_area_EducationD_TIF_2022]
EducationD_Non_TIF_list = [seperate_race_community_area_EducationD_Non_TIF_2017,seperate_race_community_area_EducationD_Non_TIF_2018,seperate_race_community_area_EducationD_Non_TIF_2019,seperate_race_community_area_EducationD_Non_TIF_2020,seperate_race_community_area_EducationD_Non_TIF_2021,seperate_race_community_area_EducationD_Non_TIF_2022]

year_race_community_EducationD_TIF  =  aggregate_MI_df_list(EducationD_TIF_list)
year_race_community_EducationD_Non_TIF  =  aggregate_MI_df_list(EducationD_Non_TIF_list)

with open(path + 'year_race_community_EducationD_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationD_TIF, file)
with open(path + 'year_race_community_EducationD_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationD_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationD_TIF.pickle', 'rb') as f:
	year_race_community_EducationD_TIF = pickle.load(f)
with open(path+'year_race_community_EducationD_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationD_Non_TIF = pickle.load(f)
year_race_community_EducationD_TIF.head(3)

,Community,High School/GED or alternative credential of White,High School/GED or alternative credential of Black,High School/GED or alternative credential of Latin
0,DOUGLAS,"[118.92564443940981, 122.66491172495085, 118.5...","[1007.0739259798419, 944.2390723804708, 849.98...","[28.571001698083503, 27.76058821998082, 26.801..."
1,OAKLAND,"[4.089083101130255, 4.88991698896581, 5.533059...","[225.78685644708614, 248.43633609443168, 271.5...","[4.261009180304999, 5.014421330060358, 5.52137..."
2,FULLER PARK,"[7.13373602693586, 8.273881542096277, 7.506313...","[465.46425170372106, 534.2164766403665, 542.04...","[30.066743378135932, 35.67174505350358, 36.669..."


In [ ]:
year_race_community_EducationD_Non_TIF

,Community,High School/GED or alternative credential of White,High School/GED or alternative credential of Black,High School/GED or alternative credential of Latin
0,DOUGLAS,"[147.5920001743116, 143.8428975312131, 132.761...","[1355.3009944039893, 1260.8890210448499, 1204....","[43.78761769954062, 41.45812069493038, 37.7593..."
1,OAKLAND,"[16.691527426154323, 15.417183080604119, 18.47...","[591.1613275899354, 605.3920218210541, 690.137...","[6.307815457069189, 7.271284532759746, 8.49961..."
2,FULLER PARK,"[2.372484458715917, 2.6171153141846433, 2.1481...","[119.44430003873808, 128.18346964338411, 121.9...","[6.4727544517519355, 7.108401665138716, 6.8908..."
3,GRAND BOULEVARD,"[11.455043762429467, 13.783251455750342, 16.12...","[365.389391585679, 409.97049881906185, 436.264...","[6.9889330859991325, 8.104653994219431, 8.6561..."
4,KENWOOD,"[202.96127346302683, 199.74287060284016, 196.6...","[1132.111583101003, 1130.0639854740375, 1239.0...","[42.55472363535344, 41.72425145905021, 40.9571..."
...,...,...,...,...
72,MOUNT GREENWOOD,"[2696.1972373381222, 2486.672324466578, 2286.0...","[135.56862913318892, 123.15848505902909, 105.5...","[218.7873188363566, 203.68725533339614, 188.57..."
73,MORGAN PARK,"[594.681903009427, 548.8129249314143, 563.1681...","[487.90415116995644, 420.8695222015582, 430.72...","[45.53140701901588, 41.65191962645272, 42.0802..."
74,OHARE,"[3564.5499999999993, 3380.1999999999994, 3255....","[109.49, 102.35, 98.15, 89.93000000000002, 100...","[848.9000000000001, 765.99, 822.7699999999999,..."
75,EDGEWATER,"[651.7329347387092, 589.7977161736977, 576.231...","[148.28959099184777, 148.67284144706616, 151.6...","[349.1413402125283, 344.22291966505594, 340.52..."


# **6.10** Education E

In [ ]:
column_name = ['Community','Some College, No Degree']

with open(path+'educationDistributionE2016.pickle', 'rb') as f:
  EducationDistributionE2016 = pickle.load(f)
one_community_area_EducationE_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionE2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2016 = seperate(one_community_area_EducationE_TIF_2016)

one_community_area_EducationE_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionE2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2016 = seperate(one_community_area_EducationE_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionE2017.pickle', 'rb') as f:
  EducationDistributionE2017 = pickle.load(f)
one_community_area_EducationE_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionE2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2017 = seperate(one_community_area_EducationE_TIF_2017)

one_community_area_EducationE_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionE2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2017 = seperate(one_community_area_EducationE_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionE2018.pickle', 'rb') as f:
  EducationDistributionE2018 = pickle.load(f)
one_community_area_EducationE_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionE2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2018 = seperate(one_community_area_EducationE_TIF_2018)

one_community_area_EducationE_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionE2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2018 = seperate(one_community_area_EducationE_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionE2019.pickle', 'rb') as f:
  EducationDistributionE2019 = pickle.load(f)
one_community_area_EducationE_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionE2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2019 = seperate(one_community_area_EducationE_TIF_2019)

one_community_area_EducationE_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionE2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2019 = seperate(one_community_area_EducationE_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionE2020.pickle', 'rb') as f:
  EducationDistributionE2020 = pickle.load(f)
one_community_area_EducationE_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionE2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2020 = seperate(one_community_area_EducationE_TIF_2020)

one_community_area_EducationE_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionE2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2020 = seperate(one_community_area_EducationE_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionE2021.pickle', 'rb') as f:
  EducationDistributionE2021 = pickle.load(f)
one_community_area_EducationE_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionE2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2021 = seperate(one_community_area_EducationE_TIF_2021)

one_community_area_EducationE_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionE2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2021 = seperate(one_community_area_EducationE_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionE2022.pickle', 'rb') as f:
  EducationDistributionE_2022 = pickle.load(f)
one_community_area_EducationE_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionE_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_TIF_2022 = seperate(one_community_area_EducationE_TIF_2022)

one_community_area_EducationE_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionE_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationE_Non_TIF_2022 = seperate(one_community_area_EducationE_Non_TIF_2022)

#######################################################################

EducationE_TIF_list = [seperate_race_community_area_EducationE_TIF_2017,seperate_race_community_area_EducationE_TIF_2018,seperate_race_community_area_EducationE_TIF_2019,seperate_race_community_area_EducationE_TIF_2020,seperate_race_community_area_EducationE_TIF_2021,seperate_race_community_area_EducationE_TIF_2022]
EducationE_Non_TIF_list = [seperate_race_community_area_EducationE_Non_TIF_2017,seperate_race_community_area_EducationE_Non_TIF_2018,seperate_race_community_area_EducationE_Non_TIF_2019,seperate_race_community_area_EducationE_Non_TIF_2020,seperate_race_community_area_EducationE_Non_TIF_2021,seperate_race_community_area_EducationE_Non_TIF_2022]

year_race_community_EducationE_TIF  =  aggregate_MI_df_list(EducationE_TIF_list)
year_race_community_EducationE_Non_TIF  =  aggregate_MI_df_list(EducationE_Non_TIF_list)

with open(path + 'year_race_community_EducationE_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationE_TIF, file)
with open(path + 'year_race_community_EducationE_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationE_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationE_TIF.pickle', 'rb') as f:
	year_race_community_EducationE_TIF = pickle.load(f)
with open(path+'year_race_community_EducationE_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationE_Non_TIF = pickle.load(f)
year_race_community_EducationE_TIF.head(3)

,Community,"Some College, No Degree of White","Some College, No Degree of Black","Some College, No Degree of Latin"
0,DOUGLAS,"[103.39596542074851, 96.0491475872212, 103.250...","[1008.9573989732372, 1025.1762514469474, 1081....","[27.07797278520531, 25.258301692178314, 27.293..."
1,OAKLAND,"[4.469509891138602, 6.037382310118537, 6.57747...","[274.7308004717507, 304.3788824596468, 334.177...","[5.370200985270589, 6.389265976044116, 7.16564..."
2,FULLER PARK,"[4.169490882727914, 3.6040685357399904, 3.7846...","[223.71418541482936, 205.84590215436427, 207.3...","[8.187349182674524, 6.808313263839436, 8.83898..."


In [ ]:
year_race_community_EducationE_Non_TIF.head(3)

,Community,"Some College, No Degree of White","Some College, No Degree of Black","Some College, No Degree of Latin"
0,DOUGLAS,"[144.97418174063856, 137.33745985813937, 145.9...","[1499.8447919409984, 1594.110300960278, 1532.4...","[42.38828418060469, 42.76687470918816, 43.2131..."
1,OAKLAND,"[14.931865211728029, 16.594769121852405, 16.91...","[616.2159138674996, 645.7247803001409, 646.354...","[6.99656465979902, 8.731894505580993, 9.139364..."
2,FULLER PARK,"[1.5664633231473013, 1.3644974185997578, 1.474...","[64.85534184311322, 61.0759274367943, 63.51645...","[2.3455929273405673, 2.1268081497626476, 2.610..."


# **6.11** Education F

In [ ]:
column_name = ['Community','Undergraduate Degree']

with open(path+'educationDistributionF2016.pickle', 'rb') as f:
  EducationDistributionF2016 = pickle.load(f)
one_community_area_EducationF_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionF2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2016 = seperate(one_community_area_EducationF_TIF_2016)

one_community_area_EducationF_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionF2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2016 = seperate(one_community_area_EducationF_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionF2017.pickle', 'rb') as f:
  EducationDistributionF2017 = pickle.load(f)
one_community_area_EducationF_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionF2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2017 = seperate(one_community_area_EducationF_TIF_2017)

one_community_area_EducationF_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionF2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2017 = seperate(one_community_area_EducationF_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionF2018.pickle', 'rb') as f:
  EducationDistributionF2018 = pickle.load(f)
one_community_area_EducationF_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionF2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2018 = seperate(one_community_area_EducationF_TIF_2018)

one_community_area_EducationF_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionF2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2018 = seperate(one_community_area_EducationF_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionF2019.pickle', 'rb') as f:
  EducationDistributionF2019 = pickle.load(f)
one_community_area_EducationF_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionF2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2019 = seperate(one_community_area_EducationF_TIF_2019)

one_community_area_EducationF_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionF2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2019 = seperate(one_community_area_EducationF_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionF2020.pickle', 'rb') as f:
  EducationDistributionF2020 = pickle.load(f)
one_community_area_EducationF_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionF2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2020 = seperate(one_community_area_EducationF_TIF_2020)

one_community_area_EducationF_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionF2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2020 = seperate(one_community_area_EducationF_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionF2021.pickle', 'rb') as f:
  EducationDistributionF2021 = pickle.load(f)
one_community_area_EducationF_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionF2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2021 = seperate(one_community_area_EducationF_TIF_2021)

one_community_area_EducationF_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionF2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2021 = seperate(one_community_area_EducationF_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionF2022.pickle', 'rb') as f:
  EducationDistributionF_2022 = pickle.load(f)
one_community_area_EducationF_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionF_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_TIF_2022 = seperate(one_community_area_EducationF_TIF_2022)

one_community_area_EducationF_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionF_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationF_Non_TIF_2022 = seperate(one_community_area_EducationF_Non_TIF_2022)

#######################################################################

EducationF_TIF_list = [seperate_race_community_area_EducationF_TIF_2017,seperate_race_community_area_EducationF_TIF_2018,seperate_race_community_area_EducationF_TIF_2019,seperate_race_community_area_EducationF_TIF_2020,seperate_race_community_area_EducationF_TIF_2021,seperate_race_community_area_EducationF_TIF_2022]
EducationF_Non_TIF_list = [seperate_race_community_area_EducationF_Non_TIF_2017,seperate_race_community_area_EducationF_Non_TIF_2018,seperate_race_community_area_EducationF_Non_TIF_2019,seperate_race_community_area_EducationF_Non_TIF_2020,seperate_race_community_area_EducationF_Non_TIF_2021,seperate_race_community_area_EducationF_Non_TIF_2022]

year_race_community_EducationF_TIF  =  aggregate_MI_df_list(EducationF_TIF_list)
year_race_community_EducationF_Non_TIF  =  aggregate_MI_df_list(EducationF_Non_TIF_list)

with open(path + 'year_race_community_EducationF_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationF_TIF, file)
with open(path + 'year_race_community_EducationF_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationF_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationF_TIF.pickle', 'rb') as f:
	year_race_community_EducationF_TIF = pickle.load(f)
with open(path+'year_race_community_EducationF_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationF_Non_TIF = pickle.load(f)
year_race_community_EducationF_TIF.head(3)

,Community,Undergraduate Degree of White,Undergraduate Degree of Black,Undergraduate Degree of Latin
0,DOUGLAS,"[156.22037067169614, 182.74685353605454, 181.1...","[1290.896588608701, 1374.8077897559253, 1248.4...","[39.47514870040557, 42.46147373039445, 40.9525..."
1,OAKLAND,"[5.209121642070653, 5.841291196761369, 6.79761...","[181.39204603146442, 217.68236177206103, 292.3...","[4.293141291150418, 5.15503024628119, 6.269643..."
2,FULLER PARK,"[4.229032818453249, 4.621673153891642, 4.83255...","[186.32284646668865, 185.54553044110858, 189.4...","[9.700188585953255, 9.472920410204003, 7.05712..."


In [ ]:
year_race_community_EducationF_Non_TIF.head(3)

,Community,Undergraduate Degree of White,Undergraduate Degree of Black,Undergraduate Degree of Latin
0,DOUGLAS,"[249.89849926036777, 280.2605764105674, 286.04...","[2138.5904173269028, 2323.0543509847485, 2305....","[64.2284243652991, 71.50904757411956, 72.07242..."
1,OAKLAND,"[18.90405492180404, 17.576566049491046, 21.022...","[592.9885927154327, 586.273675051439, 756.5143...","[8.16926628613907, 8.711240560767314, 9.571034..."
2,FULLER PARK,"[1.6495460277994032, 1.9153312355174168, 2.051...","[55.27236514824201, 61.160130954263465, 65.633...","[2.4953533647334156, 2.780027672637745, 2.5386..."


# **6.12** Education G

In [ ]:
column_name = ['Community','Graduate Degree']

with open(path+'educationDistributionG2016.pickle', 'rb') as f:
  EducationDistributionG2016 = pickle.load(f)
one_community_area_EducationG_TIF_2016 = aggregate_community_level_TIF(DemographicsDataframe2016,EducationDistributionG2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2016 = seperate(one_community_area_EducationG_TIF_2016)

one_community_area_EducationG_Non_TIF_2016 = aggregate_community_level_Non_TIF(DemographicsDataframe2016,EducationDistributionG2016,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2016 = seperate(one_community_area_EducationG_Non_TIF_2016)

#######################################################################

with open(path+'educationDistributionG2017.pickle', 'rb') as f:
  EducationDistributionG2017 = pickle.load(f)
one_community_area_EducationG_TIF_2017 = aggregate_community_level_TIF(DemographicsDataframe2017,EducationDistributionG2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2017 = seperate(one_community_area_EducationG_TIF_2017)

one_community_area_EducationG_Non_TIF_2017 = aggregate_community_level_Non_TIF(DemographicsDataframe2017,EducationDistributionG2017,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2017 = seperate(one_community_area_EducationG_Non_TIF_2017)

#######################################################################

with open(path+'educationDistributionG2018.pickle', 'rb') as f:
  EducationDistributionG2018 = pickle.load(f)
one_community_area_EducationG_TIF_2018 = aggregate_community_level_TIF(DemographicsDataframe2018,EducationDistributionG2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2018 = seperate(one_community_area_EducationG_TIF_2018)

one_community_area_EducationG_Non_TIF_2018 = aggregate_community_level_Non_TIF(DemographicsDataframe2018,EducationDistributionG2018,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2018 = seperate(one_community_area_EducationG_Non_TIF_2018)

#######################################################################

with open(path+'educationDistributionG2019.pickle', 'rb') as f:
  EducationDistributionG2019 = pickle.load(f)
one_community_area_EducationG_TIF_2019 = aggregate_community_level_TIF(DemographicsDataframe2019,EducationDistributionG2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2019 = seperate(one_community_area_EducationG_TIF_2019)

one_community_area_EducationG_Non_TIF_2019 = aggregate_community_level_Non_TIF(DemographicsDataframe2019,EducationDistributionG2019,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2019 = seperate(one_community_area_EducationG_Non_TIF_2019)

#######################################################################

with open(path+'educationDistributionG2020.pickle', 'rb') as f:
  EducationDistributionG2020 = pickle.load(f)
one_community_area_EducationG_TIF_2020 = aggregate_community_level_TIF(DemographicsDataframe2020,EducationDistributionG2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2020 = seperate(one_community_area_EducationG_TIF_2020)

one_community_area_EducationG_Non_TIF_2020 = aggregate_community_level_Non_TIF(DemographicsDataframe2020,EducationDistributionG2020,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2020 = seperate(one_community_area_EducationG_Non_TIF_2020)

#######################################################################

with open(path+'educationDistributionG2021.pickle', 'rb') as f:
  EducationDistributionG2021 = pickle.load(f)
one_community_area_EducationG_TIF_2021 = aggregate_community_level_TIF(DemographicsDataframe2021,EducationDistributionG2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2021 = seperate(one_community_area_EducationG_TIF_2021)

one_community_area_EducationG_Non_TIF_2021 = aggregate_community_level_Non_TIF(DemographicsDataframe2021,EducationDistributionG2021,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2021 = seperate(one_community_area_EducationG_Non_TIF_2021)

#######################################################################

with open(path+'educationDistributionG2022.pickle', 'rb') as f:
  EducationDistributionG_2022 = pickle.load(f)
one_community_area_EducationG_TIF_2022 = aggregate_community_level_TIF(DemographicsDataframe2022,EducationDistributionG_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_TIF_2022 = seperate(one_community_area_EducationG_TIF_2022)

one_community_area_EducationG_Non_TIF_2022 = aggregate_community_level_Non_TIF(DemographicsDataframe2022,EducationDistributionG_2022,Blockgroup_boundary, column_name)
seperate_race_community_area_EducationG_Non_TIF_2022 = seperate(one_community_area_EducationG_Non_TIF_2022)

#######################################################################

EducationG_TIF_list = [seperate_race_community_area_EducationG_TIF_2017,seperate_race_community_area_EducationG_TIF_2018,seperate_race_community_area_EducationG_TIF_2019,seperate_race_community_area_EducationG_TIF_2020,seperate_race_community_area_EducationG_TIF_2021,seperate_race_community_area_EducationG_TIF_2022]
EducationG_Non_TIF_list = [seperate_race_community_area_EducationG_Non_TIF_2017,seperate_race_community_area_EducationG_Non_TIF_2018,seperate_race_community_area_EducationG_Non_TIF_2019,seperate_race_community_area_EducationG_Non_TIF_2020,seperate_race_community_area_EducationG_Non_TIF_2021,seperate_race_community_area_EducationG_Non_TIF_2022]

year_race_community_EducationG_TIF  =  aggregate_MI_df_list(EducationG_TIF_list)
year_race_community_EducationG_Non_TIF  =  aggregate_MI_df_list(EducationG_Non_TIF_list)

with open(path + 'year_race_community_EducationG_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationG_TIF, file)
with open(path + 'year_race_community_EducationG_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_EducationG_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_EducationG_TIF.pickle', 'rb') as f:
	year_race_community_EducationG_TIF = pickle.load(f)
with open(path+'year_race_community_EducationG_Non_TIF.pickle', 'rb') as f:
	year_race_community_EducationG_Non_TIF = pickle.load(f)
year_race_community_EducationG_TIF.head(3)

,Community,Graduate Degree of White,Graduate Degree of Black,Graduate Degree of Latin
0,DOUGLAS,"[112.80028168408947, 116.9821729447278, 119.71...","[795.5094353629952, 830.5622516390613, 876.670...","[28.20960391810827, 26.772951111982643, 28.363..."
1,OAKLAND,"[3.2159324411287606, 3.320406875628254, 3.5138...","[86.52246213807271, 95.7536113646026, 114.0627...","[2.1390075093312118, 2.1736583903605005, 2.521..."
2,FULLER PARK,"[1.4015958650296463, 0.916221168491273, 0.7345...","[60.534197277564225, 33.92362049967476, 35.287...","[2.903822281272542, 1.3974404026630218, 1.4749..."


In [ ]:
year_race_community_EducationG_Non_TIF.head(3)

,Community,Graduate Degree of White,Graduate Degree of Black,Graduate Degree of Latin
0,DOUGLAS,"[188.23007614192937, 183.37751028309427, 185.4...","[1478.6833342225564, 1428.7447429762108, 1435....","[47.87624087222404, 47.51020424797048, 48.3709..."
1,OAKLAND,"[13.967381194599728, 17.511141516614657, 16.55...","[358.9894291746456, 448.3334470447616, 407.775...","[5.552551020648569, 6.276166676905514, 6.07921..."
2,FULLER PARK,"[0.6315304885286535, 0.41916231608926163, 0.33...","[23.164851292886958, 13.607111455040092, 13.42...","[1.0712101873955593, 0.5848105690247991, 0.586..."


# **6.13** Average Housing Price

In [ ]:
column_name = ['Community','Average Housing Price']


with open(path+'one_community_area_AverageHousingPrice_TIF_2017.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2017 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2017= one_community_area_AverageHousingPrice_TIF_2017.rename(columns={
    'Community Area': column_name[0],
    'Average Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2017 = seperate(one_community_area_AverageHousingPrice_TIF_2017)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2017.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2017 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2017= one_community_area_AverageHousingPrice_Non_TIF_2017.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2017 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2017)

#######################################################################

with open(path+'one_community_area_AverageHousingPrice_TIF_2018.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2018 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2018= one_community_area_AverageHousingPrice_TIF_2018.rename(columns={
    'Community Area': column_name[0],
    'Average Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2018 = seperate(one_community_area_AverageHousingPrice_TIF_2018)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2018.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2018 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2018= one_community_area_AverageHousingPrice_Non_TIF_2018.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2018 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2018)

#######################################################################

with open(path+'one_community_area_AverageHousingPrice_TIF_2019.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2019 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2019= one_community_area_AverageHousingPrice_TIF_2019.rename(columns={
    'Community Area': column_name[0],
    'Average Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2019 = seperate(one_community_area_AverageHousingPrice_TIF_2019)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2019.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2019 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2019= one_community_area_AverageHousingPrice_Non_TIF_2019.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2019 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2019)

#######################################################################

with open(path+'one_community_area_AverageHousingPrice_TIF_2020.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2020 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2020= one_community_area_AverageHousingPrice_TIF_2020.rename(columns={
    'Community Area': column_name[0],
    'Average Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2020 = seperate(one_community_area_AverageHousingPrice_TIF_2020)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2020.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2020 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2020= one_community_area_AverageHousingPrice_Non_TIF_2020.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2020 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2020)

#######################################################################

with open(path+'one_community_area_AverageHousingPrice_TIF_2021.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2021 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2021= one_community_area_AverageHousingPrice_TIF_2021.rename(columns={
    'Community Area': column_name[0],
    'Average Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2021 = seperate(one_community_area_AverageHousingPrice_TIF_2021)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2021.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2021 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2021= one_community_area_AverageHousingPrice_Non_TIF_2021.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2021 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2021)

#######################################################################

with open(path+'one_community_area_AverageHousingPrice_TIF_2022.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_TIF_2022 = pickle.load(f)
one_community_area_AverageHousingPrice_TIF_2022= one_community_area_AverageHousingPrice_TIF_2022.rename(columns={
    'Community Area': column_name[0],
    'Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_TIF_2022 = seperate(one_community_area_AverageHousingPrice_TIF_2022)

with open(path+'one_community_area_AverageHousingPrice_Non_TIF_2022.pickle', 'rb') as f:
  one_community_area_AverageHousingPrice_Non_TIF_2022 = pickle.load(f)
one_community_area_AverageHousingPrice_Non_TIF_2022= one_community_area_AverageHousingPrice_Non_TIF_2022.rename(columns={
    'Community Area': column_name[0],
    'non-TIF Racial AVG Housing Price': 'Average Housing Price'
})
seperate_race_community_area_AverageHousingPrice_Non_TIF_2022 = seperate(one_community_area_AverageHousingPrice_Non_TIF_2022)

#######################################################################

AverageHousingPrice_TIF_list = [seperate_race_community_area_AverageHousingPrice_TIF_2017,seperate_race_community_area_AverageHousingPrice_TIF_2018,seperate_race_community_area_AverageHousingPrice_TIF_2019,seperate_race_community_area_AverageHousingPrice_TIF_2020,seperate_race_community_area_AverageHousingPrice_TIF_2021,seperate_race_community_area_AverageHousingPrice_TIF_2022]
AverageHousingPrice_Non_TIF_list = [seperate_race_community_area_AverageHousingPrice_Non_TIF_2017,seperate_race_community_area_AverageHousingPrice_Non_TIF_2018,seperate_race_community_area_AverageHousingPrice_Non_TIF_2019,seperate_race_community_area_AverageHousingPrice_Non_TIF_2020,seperate_race_community_area_AverageHousingPrice_Non_TIF_2021,seperate_race_community_area_AverageHousingPrice_Non_TIF_2022]

year_race_community_AverageHousingPrice_TIF  =  aggregate_MI_df_list(AverageHousingPrice_TIF_list)
year_race_community_AverageHousingPrice_Non_TIF  =  aggregate_MI_df_list(AverageHousingPrice_Non_TIF_list)

with open(path + 'year_race_community_AverageHousingPrice_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_AverageHousingPrice_TIF, file)
with open(path + 'year_race_community_AverageHousingPrice_Non_TIF.pickle', 'wb') as file:
    pickle.dump(year_race_community_AverageHousingPrice_Non_TIF, file)

In [ ]:
with open(path+'year_race_community_AverageHousingPrice_TIF.pickle', 'rb') as f:
	year_race_community_AverageHousingPrice_TIF = pickle.load(f)
with open(path+'year_race_community_AverageHousingPrice_Non_TIF.pickle', 'rb') as f:
	year_race_community_AverageHousingPrice_Non_TIF = pickle.load(f)
year_race_community_AverageHousingPrice_TIF.head(3)

,Community,Average Housing Price of White,Average Housing Price of Black,Average Housing Price of Latin
0,DOUGLAS,"[14395726.549174283, 15565796.936483804, 13404...","[1228318.1138702654, 1380730.7046613472, 13638...","[14895218.641711487, 18268065.07695548, 196687..."
1,OAKLAND,"[6606232.274803293, 8567523.45519997, 10804062...","[180726.36338997862, 234638.19635326497, 29665...","[12964306.397061272, 17710915.428277634, 24216..."
2,FULLER PARK,"[7068487.17022147, 3746097.690132634, 4395129....","[130498.36617596762, 70225.706509598, 81911.76...","[3499859.841083292, 1778453.401249479, 2013512..."


In [ ]:
year_race_community_AverageHousingPrice_Non_TIF.head(3)

,Community,Average Housing Price of White,Average Housing Price of Black,Average Housing Price of Latin
0,DOUGLAS,"[15913346.629432127, 15407890.171548232, 12428...","[1463451.0287021035, 1396232.982766691, 123388...","[10877601.170945961, 10296769.073939994, 93624..."
1,OAKLAND,"[27850804.101855975, 33206800.561977465, 35612...","[747180.8200386224, 891632.4515527022, 959659....","[19455514.108682204, 24394625.24081586, 263811..."
2,FULLER PARK,"[2938099.5438995166, 1352118.3818425601, 16790...","[53858.515429351566, 25052.31125128027, 30969....","[1378425.5552819902, 635566.7147302014, 751565..."


# (Optional) Metrix

In [ ]:
pre_list_TIF_2017 = [seperate_race_community_area_Median_income_TIF_2017,seperate_race_community_area_Owner_TIF_2017,seperate_race_community_area_Renter_TIF_2017,seperate_race_community_area_Employment_TIF_2017,seperate_race_community_area_Unemployment_TIF_2017]
Matrix_W_TIF_2017,Matrix_B_TIF_2017,Matrix_L_TIF_2017 = Make_matric_with_many_metrics(pre_list_TIF_2017)

pre_list_Non_TIF_2017 = [seperate_race_community_area_Median_income_Non_TIF_2017,seperate_race_community_area_Owner_Non_TIF_2017,seperate_race_community_area_Renter_Non_TIF_2017,seperate_race_community_area_Employment_Non_TIF_2017,seperate_race_community_area_Unemployment_Non_TIF_2017]
Matrix_W_Non_TIF_2017,Matrix_B_Non_TIF_2017,Matrix_L_Non_TIF_2017 = Make_matric_with_many_metrics(pre_list_Non_TIF_2017)

#######################################################################

pre_list_TIF_2018 = [seperate_race_community_area_Median_income_TIF_2018,seperate_race_community_area_Owner_TIF_2018,seperate_race_community_area_Renter_TIF_2018,seperate_race_community_area_Employment_TIF_2018,seperate_race_community_area_Unemployment_TIF_2018]
Matrix_W_TIF_2018,Matrix_B_TIF_2018,Matrix_L_TIF_2018 = Make_matric_with_many_metrics(pre_list_TIF_2018)

pre_list_Non_TIF_2018 = [seperate_race_community_area_Median_income_Non_TIF_2018,seperate_race_community_area_Owner_Non_TIF_2018,seperate_race_community_area_Renter_Non_TIF_2018,seperate_race_community_area_Employment_Non_TIF_2018,seperate_race_community_area_Unemployment_Non_TIF_2018]
Matrix_W_Non_TIF_2018,Matrix_B_Non_TIF_2018,Matrix_L_Non_TIF_2018 = Make_matric_with_many_metrics(pre_list_Non_TIF_2018)

#######################################################################

pre_list_TIF_2019 = [seperate_race_community_area_Median_income_TIF_2019,seperate_race_community_area_Owner_TIF_2019,seperate_race_community_area_Renter_TIF_2019,seperate_race_community_area_Employment_TIF_2019,seperate_race_community_area_Unemployment_TIF_2019]
Matrix_W_TIF_2019,Matrix_B_TIF_2019,Matrix_L_TIF_2019 = Make_matric_with_many_metrics(pre_list_TIF_2019)

pre_list_Non_TIF_2019 = [seperate_race_community_area_Median_income_Non_TIF_2019,seperate_race_community_area_Owner_Non_TIF_2019,seperate_race_community_area_Renter_Non_TIF_2019,seperate_race_community_area_Employment_Non_TIF_2019,seperate_race_community_area_Unemployment_Non_TIF_2019]
Matrix_W_Non_TIF_2019,Matrix_B_Non_TIF_2019,Matrix_L_Non_TIF_2019 = Make_matric_with_many_metrics(pre_list_Non_TIF_2019)

#######################################################################

pre_list_TIF_2020 = [seperate_race_community_area_Median_income_TIF_2020,seperate_race_community_area_Owner_TIF_2020,seperate_race_community_area_Renter_TIF_2020,seperate_race_community_area_Employment_TIF_2020,seperate_race_community_area_Unemployment_TIF_2020]
Matrix_W_TIF_2020,Matrix_B_TIF_2020,Matrix_L_TIF_2020 = Make_matric_with_many_metrics(pre_list_TIF_2020)

pre_list_Non_TIF_2020 = [seperate_race_community_area_Median_income_Non_TIF_2020,seperate_race_community_area_Owner_Non_TIF_2020,seperate_race_community_area_Renter_Non_TIF_2020,seperate_race_community_area_Employment_Non_TIF_2020,seperate_race_community_area_Unemployment_Non_TIF_2020]
Matrix_W_Non_TIF_2020,Matrix_B_Non_TIF_2020,Matrix_L_Non_TIF_2020 = Make_matric_with_many_metrics(pre_list_Non_TIF_2020)

#######################################################################

pre_list_TIF_2021 = [seperate_race_community_area_Median_income_TIF_2021,seperate_race_community_area_Owner_TIF_2021,seperate_race_community_area_Renter_TIF_2021,seperate_race_community_area_Employment_TIF_2021,seperate_race_community_area_Unemployment_TIF_2021]
Matrix_W_TIF_2021,Matrix_B_TIF_2021,Matrix_L_TIF_2021 = Make_matric_with_many_metrics(pre_list_TIF_2021)

pre_list_Non_TIF_2021 = [seperate_race_community_area_Median_income_Non_TIF_2021,seperate_race_community_area_Owner_Non_TIF_2021,seperate_race_community_area_Renter_Non_TIF_2021,seperate_race_community_area_Employment_Non_TIF_2021,seperate_race_community_area_Unemployment_Non_TIF_2021]
Matrix_W_Non_TIF_2021,Matrix_B_Non_TIF_2021,Matrix_L_Non_TIF_2021 = Make_matric_with_many_metrics(pre_list_Non_TIF_2021)

#######################################################################

pre_list_TIF_2022 = [seperate_race_community_area_Median_income_TIF_2022,seperate_race_community_area_Owner_TIF_2022,seperate_race_community_area_Renter_TIF_2022,seperate_race_community_area_Employment_TIF_2022,seperate_race_community_area_Unemployment_TIF_2022]
Matrix_W_TIF_2022,Matrix_B_TIF_2022,Matrix_L_TIF_2022 = Make_matric_with_many_metrics(pre_list_TIF_2022)

pre_list_Non_TIF_2022 = [seperate_race_community_area_Median_income_Non_TIF_2022,seperate_race_community_area_Owner_Non_TIF_2022,seperate_race_community_area_Renter_Non_TIF_2022,seperate_race_community_area_Employment_Non_TIF_2022,seperate_race_community_area_Unemployment_Non_TIF_2022]
Matrix_W_Non_TIF_2022,Matrix_B_Non_TIF_2022,Matrix_L_Non_TIF_2022 = Make_matric_with_many_metrics(pre_list_Non_TIF_2022)

#######################################################################


In [ ]:

with open(path + 'Matrix_W_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2017, file)
with open(path + 'Matrix_W_Non_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2017, file)

with open(path + 'Matrix_B_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2017, file)
with open(path + 'Matrix_B_Non_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2017, file)

with open(path + 'Matrix_L_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2017, file)
with open(path + 'Matrix_L_Non_TIF_2017.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2017, file)

#######################################################################

with open(path + 'Matrix_W_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2018, file)
with open(path + 'Matrix_W_Non_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2018, file)

with open(path + 'Matrix_B_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2018, file)
with open(path + 'Matrix_B_Non_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2018, file)

with open(path + 'Matrix_L_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2018, file)
with open(path + 'Matrix_L_Non_TIF_2018.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2018, file)

#######################################################################

with open(path + 'Matrix_W_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2019, file)
with open(path + 'Matrix_W_Non_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2019, file)

with open(path + 'Matrix_B_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2019, file)
with open(path + 'Matrix_B_Non_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2019, file)

with open(path + 'Matrix_L_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2019, file)
with open(path + 'Matrix_L_Non_TIF_2019.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2019, file)

#######################################################################

with open(path + 'Matrix_W_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2020, file)
with open(path + 'Matrix_W_Non_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2020, file)

with open(path + 'Matrix_B_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2020, file)
with open(path + 'Matrix_B_Non_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2020, file)

with open(path + 'Matrix_L_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2020, file)
with open(path + 'Matrix_L_Non_TIF_2020.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2020, file)

#######################################################################

with open(path + 'Matrix_W_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2021, file)
with open(path + 'Matrix_W_Non_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2021, file)

with open(path + 'Matrix_B_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2021, file)
with open(path + 'Matrix_B_Non_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2021, file)

with open(path + 'Matrix_L_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2021, file)
with open(path + 'Matrix_L_Non_TIF_2021.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2021, file)

#######################################################################

with open(path + 'Matrix_W_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_W_TIF_2022, file)
with open(path + 'Matrix_W_Non_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_W_Non_TIF_2022, file)

with open(path + 'Matrix_B_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_B_TIF_2022, file)
with open(path + 'Matrix_B_Non_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_B_Non_TIF_2022, file)

with open(path + 'Matrix_L_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_L_TIF_2022, file)
with open(path + 'Matrix_L_Non_TIF_2022.pickle', 'wb') as file:
    pickle.dump(Matrix_L_Non_TIF_2022, file)


In [ ]:
Matrix_W_TIF_2017['Median_income']

,Community,Median income,Owner,Renter,Employment,Unemployment
0,DOUGLAS,32500,79.531382,192.927478,464.647572,658.265518
1,OAKLAND,27500,4.795101,10.938704,17.728943,8.144041
2,FULLER PARK,22500,5.174791,8.960021,14.836078,15.216455
3,GRAND BOULEVARD,32500,61.386994,141.447323,224.405853,133.809584
4,KENWOOD,32500,33.088986,50.930098,66.816422,48.699451
...,...,...,...,...,...,...
72,MOUNT GREENWOOD,67500,258.582432,63.041673,479.543405,202.852888
73,MORGAN PARK,55000,568.814355,294.403621,1145.473763,568.372954
74,OHARE,55000,568.814355,294.403621,1145.473763,568.372954
75,EDGEWATER,47500,6163.501175,9430.319594,18265.848614,9163.509846


In [ ]:
Matrix_W_Non_TIF_2017

,Community,Median income,Owner,Renter,Employment,Unemployment
0,DOUGLAS,37500,122.372700,346.743857,610.178716,738.698573
1,OAKLAND,27500,14.384570,43.577663,65.560615,33.003162
2,FULLER PARK,22500,2.028198,3.220518,5.784217,5.554726
3,GRAND BOULEVARD,32500,11.260361,30.437479,46.734325,26.499166
4,KENWOOD,47500,618.210377,1131.912749,1791.424115,1131.753790
...,...,...,...,...,...,...
72,MOUNT GREENWOOD,67500,4726.638238,761.872259,8217.162296,4426.007817
73,MORGAN PARK,67500,1387.827186,456.208152,2557.233917,1286.094194
74,OHARE,47500,3730.860682,4110.836732,9687.247614,5105.811152
75,EDGEWATER,47500,1208.976846,1829.749305,3898.323297,1455.071748


In [ ]:
a = pd.read_csv("/content/drive/MyDrive/IPRO497_Demographic Team/Final/data/housing_price/ACSDT5Y2022.B25075-Data.csv")
b = pd.read_csv('/content/drive/MyDrive/IPRO497_Demographic Team/Final/data/housing_price/ACSDT5Y2017.B25075-Data.csv')

a.iloc[0]

GEO_ID                                                 Geography
NAME                                        Geographic Area Name
B25075_001E                                     Estimate!!Total:
B25075_001M                              Margin of Error!!Total:
B25075_002E                  Estimate!!Total:!!Less than $10,000
B25075_002M           Margin of Error!!Total:!!Less than $10,000
B25075_003E                 Estimate!!Total:!!$10,000 to $14,999
B25075_003M          Margin of Error!!Total:!!$10,000 to $14,999
B25075_004E                 Estimate!!Total:!!$15,000 to $19,999
B25075_004M          Margin of Error!!Total:!!$15,000 to $19,999
B25075_005E                 Estimate!!Total:!!$20,000 to $24,999
B25075_005M          Margin of Error!!Total:!!$20,000 to $24,999
B25075_006E                 Estimate!!Total:!!$25,000 to $29,999
B25075_006M          Margin of Error!!Total:!!$25,000 to $29,999
B25075_007E                 Estimate!!Total:!!$30,000 to $34,999
B25075_007M          Marg

In [ ]:
b.iloc[0]

GEO_ID                                                Geography
NAME                                       Geographic Area Name
B25075_001E                                     Estimate!!Total
B25075_001M                              Margin of Error!!Total
B25075_002E                  Estimate!!Total!!Less than $10,000
B25075_002M           Margin of Error!!Total!!Less than $10,000
B25075_003E                 Estimate!!Total!!$10,000 to $14,999
B25075_003M          Margin of Error!!Total!!$10,000 to $14,999
B25075_004E                 Estimate!!Total!!$15,000 to $19,999
B25075_004M          Margin of Error!!Total!!$15,000 to $19,999
B25075_005E                 Estimate!!Total!!$20,000 to $24,999
B25075_005M          Margin of Error!!Total!!$20,000 to $24,999
B25075_006E                 Estimate!!Total!!$25,000 to $29,999
B25075_006M          Margin of Error!!Total!!$25,000 to $29,999
B25075_007E                 Estimate!!Total!!$30,000 to $34,999
B25075_007M          Margin of Error!!To